<a href="https://colab.research.google.com/github/hassaku12/manabiDX2025/blob/main/%E6%BC%94%E7%BF%9203%EF%BC%88%E8%A9%A6%E3%81%99%E7%94%A8%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
AUC0.7以上を目指すため、現状分析を再確認し、新しい特徴量のアイデア出し、実装、モデル学習と評価、特徴量の選択と調整、アンサンブルモデルの調整、最終予測と提出ファイルの作成を行います。

## 現状分析の再確認

### Subtask:
現在の分析結果（資産規模別購入率、投資経験別購入率、損益状況別購入率、投資方針×年齢層の組み合わせ、時期別購入率、投資効率別購入率など）を改めて確認し、特に購入率に大きな差が出ている顧客層や傾向を特定します。


In [36]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the base directory (re-defining in case the environment was reset)
base_dir = '/content/drive/MyDrive/Colab Notebooks/マナビDX'

# Import necessary libraries (re-importing as a safeguard)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib # Import again just in case

# Re-load the dataframes
print("📊 データ読み込み中...")
train_df = pd.read_csv(base_dir + '/train.csv')
test_df = pd.read_csv(base_dir + '/test.csv')
submission_df = pd.read_csv(base_dir + '/sample_submit.csv', header=None)
assessment_df = pd.read_csv(base_dir + '/適合性判定シート一覧表.csv')

# Re-load additional data if available
print("📊 追加データ読み込み中...")
try:
    trading_df = pd.read_csv(base_dir + '/約定データ一覧表.csv')
    product_df = pd.read_csv(base_dir + '/商品リスト.csv')
    HAS_ALL_DATA = True
except Exception as e:
    print(f"❌ データ読み込みエラー: {e}")
    HAS_ALL_DATA = False

# Re-run preprocessing steps
print("🧹 データ前処理中...")
assessment_df['取引日'] = pd.to_datetime(assessment_df['取引日'])
assessment_df_latest = assessment_df.sort_values(['顧客ID', '取引日']).drop_duplicates(subset='顧客ID', keep='last')
assessment_df_selected = assessment_df_latest[['顧客ID', '顧客年齢', '投資経験（株式）']]

train_df = pd.merge(train_df, assessment_df_selected, on='顧客ID', how='left')
test_df = pd.merge(test_df, assessment_df_selected, on='顧客ID', how='left')

train_df['顧客年齢'] = train_df['顧客年齢'].fillna(train_df['顧客年齢'].mean())
test_df['顧客年齢'] = test_df['顧客年齢'].fillna(test_df['顧客年齢'].mean())
train_df['投資経験（株式）'] = train_df['投資経験（株式）'].fillna(0)
test_df['投資経験（株式）'] = test_df['投資経験（株式）'].fillna(0)

train_df['year'] = train_df['基準年月'].str.split('-').str[0].astype(int)
train_df['month'] = train_df['基準年月'].str.split('-').str[1].astype(int)
test_df['year'] = test_df['基準年月'].str.split('-').str[0].astype(int)
test_df['month'] = test_df['基準年月'].str.split('-').str[1].astype(int)

# Re-run the analysis cell to regenerate variables
print("📊 再度詳細分析を実行中...")

# 1. 資産規模による購入率の違い
print("\n💰 資産規模別購入率:")
# Check if the column exists before creating it
if '資産規模_区分' not in train_df.columns:
    train_df['資産規模_区分'] = pd.cut(train_df['時価価額'],
                                    bins=[0, 100000, 500000, 1000000, 5000000, float('inf')],
                                    labels=['~10万', '10-50万', '50-100万', '100-500万', '500万~'],
                                    right=False) # Added right=False for inclusivity of lower bound

asset_purchase_rate = train_df.groupby('資産規模_区分')['y'].agg(['count', 'sum', 'mean']).reset_index()
asset_purchase_rate.columns = ['資産規模_区分', '顧客数', '購入者数', '購入率']
asset_purchase_rate['購入率'] = asset_purchase_rate['購入率'] * 100
display(asset_purchase_rate)

# 2. 投資経験による影響
print("\n📈 投資経験別購入率:")
experience_purchase = train_df.groupby('投資経験（株式）')['y'].agg(['count', 'mean']).reset_index()
experience_purchase.columns = ['投資経験（株式）', '顧客数', '購入率']
experience_purchase['購入率'] = experience_purchase['購入率'] * 100
display(experience_purchase.head(10))

# 3. 損益状況による違い
print("\n📊 損益状況別購入率:")
# Check if the column exists before creating it
if '損益状況' not in train_df.columns:
    train_df['損益状況'] = 'その他'
    train_df.loc[train_df['評価損益'] < -50000, '損益状況'] = '大幅損失'
    train_df.loc[(train_df['評価損益'] >= -50000) & (train_df['評価損益'] < 0), '損益状況'] = '軽微損失'
    train_df.loc[(train_df['評価損益'] >= 0) & (train_df['評価損益'] < 50000), '損益状況'] = '軽微利益'
    train_df.loc[train_df['評価損益'] >= 50000, '損益状況'] = '大幅利益'

profit_loss_purchase = train_df.groupby('損益状況')['y'].agg(['count', 'sum', 'mean']).reset_index()
profit_loss_purchase.columns = ['損益状況', '顧客数', '購入者数', '購入率']
profit_loss_purchase['購入率'] = profit_loss_purchase['購入率'] * 100
display(profit_loss_purchase)

# 4. 投資方針×年齢層の組み合わせ効果
print("\n🎯 投資方針×年齢層の組み合わせ:")
# Check if the column exists before creating it
if '年齢層' not in train_df.columns:
    train_df['年齢層'] = pd.cut(train_df['顧客年齢'],
                           bins=[0, 40, 50, 60, 70, 100],
                           labels=['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~'])

combination_analysis = train_df.groupby(['年齢層', '投資方針'])['y'].agg(['count', 'mean']).reset_index()
combination_analysis.columns = ['年齢層', '投資方針', '顧客数', '購入率']
combination_analysis['購入率'] = combination_analysis['購入率'] * 100
display(combination_analysis)

# 5. 時期（Term）による購入率の変化
print("\n📅 時期別購入率:")
term_purchase_rates = []
for term_id in range(1, 7):
    term_data = train_df[train_df[f'train_term_{term_id}'] == 1]
    purchase_rate = term_data['y'].mean() * 100
    term_purchase_rates.append({
        'Term': term_id,
        '顧客数': len(term_data),
        '購入率': purchase_rate
    })

term_df = pd.DataFrame(term_purchase_rates)
display(term_df)

# 6. 投資効率レンジ別の詳細分析
print("\n🏪 投資効率レンジ別の詳細分析:")
# Check if the column exists before creating it
if '投資効率' not in train_df.columns:
     train_df['投資効率'] = train_df['時価価額'] / (train_df['取得価額'] + 1)

efficiency_analysis = train_df.groupby(pd.cut(train_df['投資効率'],
                                             bins=[0, 0.8, 0.95, 1.05, 1.2, float('inf')],
                                             labels=['大幅損失', '軽微損失', '横ばい', '軽微利益', '大幅利益']))['y'].agg(['count', 'sum', 'mean']).reset_index()
efficiency_analysis.columns = ['投資効率レンジ', '顧客数', '購入者数', '購入率']
efficiency_analysis['購入率'] = efficiency_analysis['購入率'] * 100
display(efficiency_analysis)

# Now summarize the findings after successful execution
print("\n--- 分析サマリー ---")

print("\n1. 資産規模別購入率:")
display(asset_purchase_rate)
print("\n観察: '〜10万' の資産規模区分が高い購入率（19.89%）を示しています。他のカテゴリのデータは分析に含まれていないため、ビニング範囲またはデータ分布に潜在的な問題があることを示唆しています。『〜10万』がより低い資産規模を表していると仮定すると、資産規模の小さい顧客が購入しやすい傾向があることを示しています。")

print("\n2. 投資経験別購入率:")
display(experience_purchase.head(10))
print("\n観察: 投資経験のある顧客（'投資経験（株式）' > 0）は、経験のない顧客（18.56%）と比較して購入率が高くなっています（23.15%）。投資経験は肯定的な予測因子です。")

print("\n3. 損益状況別購入率:")
display(profit_loss_purchase)
print("\n観察: '大幅損失' (27.16%)、'軽微損失' (25.31%)、および '大幅利益' (25.83%) のカテゴリの顧客は、'軽微利益' (18.83%) または 'その他' の顧客よりも高い購入率を示しています。これは興味深い結果であり、損益の極端な状況、または損失の状況にある顧客が購入しやすい傾向があることを示唆しています。")

print("\n4. 投資方針×年齢層の組み合わせ:")
display(combination_analysis)
print("\n観察: 若年層（〜40歳、40-50歳）は概して購入率が高くなっています。『投資方針 3』は、ほとんどの年齢層で一貫して高い購入率を示しています。50-60歳の年齢層は、投資方針に関係なく購入率が著しく低くなっています。")

print("\n5. 時期別購入率:")
display(term_df)
print("\n観察: 購入率は異なる Term 間で比較的安定しています（約19.7%〜19.9%）。Term ID に基づく大きな変動はありません。")

print("\n6. 投資効率レンジ別の詳細分析:")
display(efficiency_analysis)
print("\n観察: 損益と同様に、『大幅損失』（27.16%）、『大幅利益』（25.83%）、および『横ばい』（25.78%）の投資効率レンジの顧客は、『軽微損失』（22.12%）および『軽微利益』（25.48%）と比較して高い購入率を示しています。これは、極端なパフォーマンスまたはブレークイーブンの状況が購入を促進する可能性があるという考えを裏付けています。")

print("\n--- 全体的な結論 ---")
print("購入率が高い主要な顧客セグメントは以下の通りです。")
print("- 投資経験のある顧客。")
print("- 現在、大幅な損失または利益の状況にある顧客、あるいはブレークイーブンの顧客。")
print("- 若年層（50歳未満）および『投資方針 3』の顧客。")
print("50-60歳の年齢層の顧客は購入率が低い傾向があります。")
print("資産規模の分析は、データ分布またはビニングの問題により、さらに調査が必要です。")
print("購入率は時間（Term）に対して安定しているように見えます。")
print("\nこれらの洞察は、単純なデモグラフィック要因だけでなく、過去の経験、現在のポートフォリオパフォーマンス、投資アプローチなどの要因が購入意思決定に大きく影響していることを示唆しています。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📊 データ読み込み中...
📊 追加データ読み込み中...
🧹 データ前処理中...
📊 再度詳細分析を実行中...

💰 資産規模別購入率:


,資産規模_区分,顧客数,購入者数,購入率
0,~10万,90000,17899,19.887778
1,10-50万,0,0,NaN
2,50-100万,0,0,NaN
3,100-500万,0,0,NaN
4,500万~,0,0,NaN



📈 投資経験別購入率:


,投資経験（株式）,顧客数,購入率
0,0.0,63990,18.560713
1,1.0,26010,23.152634



📊 損益状況別購入率:


...


観察: 若年層（〜40歳、40-50歳）は概して購入率が高くなっています。『投資方針 3』は、ほとんどの年齢層で一貫して高い購入率を示しています。50-60歳の年齢層は、投資方針に関係なく購入率が著しく低くなっています。

5. 時期別購入率:


,Term,顧客数,購入率
0,1,80000,19.722500
1,2,82000,19.760976
2,3,84000,19.803571
3,4,86000,19.845349
4,5,88000,19.886364
5,6,90000,19.887778



観察: 購入率は異なる Term 間で比較的安定しています（約19.7%〜19.9%）。Term ID に基づく大きな変動はありません。

6. 投資効率レンジ別の詳細分析:


,投資効率レンジ,顧客数,購入者数,購入率
0,大幅損失,394,107,27.157360
1,軽微損失,5103,1129,22.124241
2,横ばい,38566,9942,25.779184
3,軽微利益,16752,4269,25.483524
4,大幅利益,9285,2398,25.826602



観察: 損益と同様に、『大幅損失』（27.16%）、『大幅利益』（25.83%）、および『横ばい』（25.78%）の投資効率レンジの顧客は、『軽微損失』（22.12%）および『軽微利益』（25.48%）と比較して高い購入率を示しています。これは、極端なパフォーマンスまたはブレークイーブンの状況が購入を促進する可能性があるという考えを裏付けています。

--- 全体的な結論 ---
購入率が高い主要な顧客セグメントは以下の通りです。
- 投資経験のある顧客。
- 現在、大幅な損失または利益の状況にある顧客、あるいはブレークイーブンの顧客。
- 若年層（50歳未満）および『投資方針 3』の顧客。
50-60歳の年齢層の顧客は購入率が低い傾向があります。
資産規模の分析は、データ分布またはビニングの問題により、さらに調査が必要です。
購入率は時間（Term）に対して安定しているように見えます。

これらの洞察は、単純なデモグラフィック要因だけでなく、過去の経験、現在のポートフォリオパフォーマンス、投資アプローチなどの要因が購入意思決定に大きく影響していることを示唆しています。


## 新しい特徴量のアイデア出し

### Subtask:
特定した顧客層や傾向に基づき、さらにモデルの予測力を高める可能性のある特徴量のアイデアを具体的にリストアップします。例えば、過去の取引データや商品リストをより詳細に分析して得られる情報や、既存の特徴量を組み合わせた交互作用項などが考えられます。


In [37]:
# 1. Identify key trends from previous analysis:
#    - Investment experience matters. # 投資経験は重要です。
#    - Profit/Loss and Investment Efficiency extremes (loss or high gain/break-even) matter. # 損益および投資効率の極端な状況（損失または高利益/ブレークイーブン）は重要です。
#    - Age group (younger generally higher, 50-60 lower) and Investment Policy (Policy 3 higher) matter. # 年齢層（若年層は概して高く、50-60歳は低い）および投資方針（方針3が高い）は重要です。
#    - Asset size analysis was inconclusive and needs better handling. # 資産規模の分析は決定的ではなく、より良い処理が必要です。
#    - Term doesn't seem to have a strong impact alone. # Term 単独では強い影響はないようです。

# 2. Brainstorm new features based on these trends and available data (trading_df, product_df): # これらの傾向および利用可能なデータ（trading_df, product_df）に基づき、新しい特徴量をブレインストーミングします。

new_feature_ideas = [
    # Features from Trading Data (more detailed behavior) # 取引データからの特徴量（より詳細な行動）
    {"name": "過去取引頻度_年", "description": "過去の総取引回数を年数で割ったもの。アクティブな顧客ほど高い。", "source": "trading_df"},
    {"name": "直近取引からの日数", "description": "最後の取引日からの経過日数。最近取引した顧客は再度購入しやすいか。", "source": "trading_df"},
    {"name": "特定カテゴリ投資比率", "description": "特定の高リスク/高リターンカテゴリ（例: 外国株式、高利回り債券）への投資額比率。リスク志向をより詳細に捉える。", "source": "trading_df, product_df"},
    {"name": "損益額変動性", "description": "過去の売却・償還損益の標準偏差。損益のブレが大きい顧客はリスク許容度が高いか。", "source": "trading_df"},
    {"name": "平均保有期間", "description": "各商品の取得日から売却/償還日までの平均期間。短期志向か長期志向か。", "source": "trading_df"}, # Requires linking trades of the same product # 同じ商品の取引を紐付ける必要があります
    {"name": "チャネル別取引比率", "description": "オンライン取引と対面取引の比率。デジタル利用度をより詳細に。", "source": "trading_df"},

    # Features from Product Data (current portfolio characteristics) # 商品データからの特徴量（現在のポートフォリオ特性）
    {"name": "保有商品数_カテゴリ別", "description": "保有している商品カテゴリの数。商品多様性をより詳細に。", "source": "trading_df, product_df"}, # Re-evaluating existing idea with more detail # 既存のアイデアをより詳細に再評価
    {"name": "保有商品_平均リスクカテゴリ", "description": "保有商品のリスクカテゴリの平均値（数値化後）。ポートフォリオ全体のリスクレベル。", "source": "trading_df, product_df"},
    {"name": "保有商品_平均償還日までの期間", "description": "保有商品の償還日までの平均期間（あれば）。長期保有志向か。", "source": "trading_df, product_df"},

    # Interaction and Transformation Features (combining existing info) # 交互作用および変換特徴量（既存情報の組み合わせ）
    {"name": "顧客年齢_二乗", "description": "顧客年齢の非線形効果を捉える。", "source": "train_df, test_df (transformed)"},
    {"name": "時価価額_対数", "description": "資産規模の歪んだ分布を正規化し、影響を滑らかにする。", "source": "train_df, test_df (transformed)"},
    {"name": "投資経験×損益状況", "description": "投資経験と損益状況の組み合わせ効果。経験者は損失をどう受け止めるか。", "source": "train_df, test_df (interaction)"},
    {"name": "年齢層×損益状況", "description": "年齢層と損益状況の組み合わせ効果。", "source": "train_df, test_df (interaction)"},
    {"name": "投資方針×投資経験", "description": "投資方針と投資経験の組み合わせ効果。", "source": "train_df, test_df (interaction)"},
    {"name": "資産規模_区分_ダミー", "description": "資産規模区分をダミー変数化。分析で示された違いをモデルに学習させる。", "source": "train_df, test_df (dummy)"},

    # Behavioral/Inferred Features # 行動/推測特徴量
    {"name": "損失回復期待フラグ", "description": "大幅損失顧客が、損失を取り戻そうとして購入する傾向があるか。", "source": "train_df, test_df (inferred from 損益状況)"}, # 損益状況から推測
    {"name": "利益確定後再投資フラグ", "description": "大幅利益顧客が、利益を確定して別の商品に再投資する傾向があるか。", "source": "train_df, test_df (inferred from 損益状況)"}, # 損益状況から推測
]

# 3. Print the list of ideas # アイデアリストを表示
print("### 新しい特徴量のアイデア ###")
for i, idea in enumerate(new_feature_ideas):
    print(f"{i+1}. {idea['name']}: {idea['description']} (データソース: {idea['source']})")

print("\nこれらの特徴量は、過去の分析で示された顧客の行動、ポートフォリオ特性、既存特徴量の複雑な関係性を捉えることで、モデルの予測力向上に貢献する可能性があります。特に、過去の取引詳細や商品情報を活用した特徴量は、顧客一人ひとりの投資プロファイルをより深く理解するために有効です。")

### 新しい特徴量のアイデア ###
1. 過去取引頻度_年: 過去の総取引回数を年数で割ったもの。アクティブな顧客ほど高い。 (データソース: trading_df)
2. 直近取引からの日数: 最後の取引日からの経過日数。最近取引した顧客は再度購入しやすいか。 (データソース: trading_df)
3. 特定カテゴリ投資比率: 特定の高リスク/高リターンカテゴリ（例: 外国株式、高利回り債券）への投資額比率。リスク志向をより詳細に捉える。 (データソース: trading_df, product_df)
4. 損益額変動性: 過去の売却・償還損益の標準偏差。損益のブレが大きい顧客はリスク許容度が高いか。 (データソース: trading_df)
5. 平均保有期間: 各商品の取得日から売却/償還日までの平均期間。短期志向か長期志向か。 (データソース: trading_df)
6. チャネル別取引比率: オンライン取引と対面取引の比率。デジタル利用度をより詳細に。 (データソース: trading_df)
7. 保有商品数_カテゴリ別: 保有している商品カテゴリの数。商品多様性をより詳細に。 (データソース: trading_df, product_df)
8. 保有商品_平均リスクカテゴリ: 保有商品のリスクカテゴリの平均値（数値化後）。ポートフォリオ全体のリスクレベル。 (データソース: trading_df, product_df)
9. 保有商品_平均償還日までの期間: 保有商品の償還日までの平均期間（あれば）。長期保有志向か。 (データソース: trading_df, product_df)
10. 顧客年齢_二乗: 顧客年齢の非線形効果を捉える。 (データソース: train_df, test_df (transformed))
11. 時価価額_対数: 資産規模の歪んだ分布を正規化し、影響を滑らかにする。 (データソース: train_df, test_df (transformed))
12. 投資経験×損益状況: 投資経験と損益状況の組み合わせ効果。経験者は損失をどう受け止めるか。 (データソース: train_df, test_df (interaction))
13. 年齢層×損益状況: 年齢層と損益状況の組み合わせ効果。 (データソ

## 特徴量の実装

### Subtask:
新しい特徴量のアイデア出しでリストアップした新しい特徴量をコードとして実装します。既存の`create_final_features`関数を修正・拡張して、これらの新しい特徴量を追加できるようにします。


In [38]:
# Copy and rename the existing function
def create_advanced_features(df):
    """拡張版特徴量作成（新しいアイデアを含む）"""
    df = df.copy()

    # --- Helper Functions (Nested to keep them local to this feature creation process) ---
    # Re-define add_dummy_trading_features
    def add_dummy_trading_features(df):
        """ダミー取引特徴量追加"""
        dummy_trading_cols = [
            '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計',
            '売却回数', '償還損益合計', '償還回数', 'デジタル利用率',
            '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益',
            '投資成功体験', '平均利益率', 'リスク管理度', '売却損益変動性',
            '償還損益変動性', '総損益変動性', '直近取引からの日数' # Add new dummy cols
        ]
        for feature in dummy_trading_cols:
             if feature not in df.columns:
                 if feature == '直近取引からの日数':
                     df[feature] = 365 * 10 # Large number for recency if no data
                 else:
                     df[feature] = 0

        if '投資家タイプ' not in df.columns:
            df['投資家タイプ'] = 'その他'

        return df

    # Re-define add_dummy_product_features
    def add_dummy_product_features(df):
         """ダミー商品特徴量追加"""
         dummy_product_cols = [
             '主要商品カテゴリ', '主要リスクカテゴリ', '円建て比率', '留意商品比率',
             'リスク指向度', '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数' # Add new dummy cols
         ]
         for feature in dummy_product_cols:
             if feature not in df.columns:
                 if feature in ['主要商品カテゴリ', '主要リスクカテゴリ']:
                     df[feature] = 'その他' if feature == '主要商品カテゴリ' else 'Medium'
                 elif feature == '最終償還日までの日数':
                     df[feature] = 365 * 10 # Large number for recency if no data
                 else:
                     df[feature] = 0.0

         return df

    # Adjust add_dummy_features to check for column existence and cover new dummies
    def add_dummy_features_adjusted(df):
        """ダミー特徴量追加 (既存カラムを上書きしない)"""
        dummy_features = [
            '過去取引回数', '過去累積損益', '投資成功体験', 'デジタル利用率',
            '商品多様性', '主要リスクカテゴリ', '投資家タイプ', 'リスク管理度', 'リスク指向度',
            # Add new dummy features here to ensure they are covered if HAS_ALL_DATA is False and specific dummy functions aren't called
            '売却損益変動性', '償還損益変動性', '総損益変動性', '直近取引からの日数',
            '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数',
            '円建て比率', '留意商品比率' # Ensure these base features are covered
        ]
        for feature in dummy_features:
            if feature not in df.columns: # Only add if column doesn't exist
                if feature in ['主要リスクカテゴリ', '投資家タイプ', '主要商品カテゴリ']: # Added 主要商品カテゴリ
                    df[feature] = 'その他' if feature == '投資家タイプ' else 'Medium' if feature == '主要リスクカテゴリ' else 'その他'
                elif feature == '直近取引からの日数' or feature == '最終償還日までの日数':
                     df[feature] = 365 * 10 # Large number for recency/maturity if no data
                elif feature in ['円建て比率', '留意商品比率', '平均リスクカテゴリ_数値', '保有商品数_ユニーク']:
                     df[feature] = 0.0
                else:
                    df[feature] = 0 # Default to 0 for other numeric dummies

        return df

    # Re-define add_real_trading_features with new features
    def add_real_trading_features_v2(df):
        """実際の約定データから特徴量作成 (v2 - 新機能追加)"""
        try:
            trading_df_copy = trading_df.copy() # Work on a copy to avoid modifying global df
            trading_df_copy['取引日'] = pd.to_datetime(trading_df_copy['取引日'])
            # Filter trading data up to a certain point relative to the analysis date if possible,
            # or use a fixed past window (e.g., up to '2021-11-30' for train data aggregation)
            # For testing train_df and test_df directly with this function outside the pipeline context,
            # we need to use a date appropriate for the data's timestamp.
            # However, the pipeline aggregates per term based on train_term_X flags.
            # Let's keep the fixed date logic for now, assuming this function is called
            # within a context that handles time slicing, or accepts a cut-off date.
            # For this general feature creation function, a fixed date (like the train data end date eve) is safer.
            analysis_date_for_aggregation = pd.to_datetime('2021-11-30') # Using a fixed date before test set

            past_trading = trading_df_copy[trading_df_copy['取引日'] <= analysis_date_for_aggregation]

            customer_stats = past_trading.groupby('顧客ID').agg({
                '取得価額': ['count', 'sum', 'mean'],
                '売却損益': ['sum', 'count', 'std'], # Added std for volatility
                '償還損益': ['sum', 'count', 'std'], # Added std for volatility
                'オンライン取引フラグ': 'mean',
                '商品名': 'nunique',
                'ゴール設定実施': 'mean',
                'ロスカット設定実施': 'mean',
                '取引日': 'max' # Added max date for recency
            }).fillna(0)

            # Flatten multi-level columns
            customer_stats.columns = ['_'.join(col).strip('_') for col in customer_stats.columns.values]

            customer_stats = customer_stats.rename(columns={
                '取得価額_count': '過去取引回数',
                '取得価額_sum': '過去総取得額',
                '取得価額_mean': '過去平均取得額',
                '売却損益_sum': '売却損益合計',
                '売却損益_count': '売却回数',
                '売却損益_std': '売却損益変動性', # New
                '償還損益_sum': '償還損益合計',
                '償還損益_count': '償還回数',
                '償還損益_std': '償還損益変動性', # New
                'オンライン取引フラグ_mean': 'デジタル利用率',
                '商品名_nunique': '商品多様性',
                'ゴール設定実施_mean': 'ゴール設定率',
                'ロスカット設定実施_mean': 'ロスカット設定率',
                '取引日_max': '最終取引日' # New
            })

            customer_stats['過去累積損益'] = customer_stats['売却損益合計'] + customer_stats['償還損益合計']
            customer_stats['投資成功体験'] = (customer_stats['過去累積損益'] > 0).astype(int)
            customer_stats['平均利益率'] = np.where((customer_stats['過去総取得額'] + 1) != 0,
                                                 customer_stats['過去累積損益'] / (customer_stats['過去総取得額'] + 1), 0)

            customer_stats['リスク管理度'] = (customer_stats['ゴール設定率'] + customer_stats['ロスカット設定率']) / 2.0 # Use 2.0 for float division

            # New: Total損益変動性 - Fill NaN std with 0 before summing/averaging
            customer_stats['売却損益変動性'] = customer_stats['売却損益変動性'].fillna(0)
            customer_stats['償還損益変動性'] = customer_stats['償還損益変動性'].fillna(0)
            customer_stats['総損益変動性'] = customer_stats[['売却損益変動性', '償還損益変動性']].mean(axis=1)


            # New: Recency (Days since last trade relative to analysis date)
            customer_stats['直近取引からの日数'] = (analysis_date_for_aggregation - customer_stats['最終取引日']).dt.days.fillna(365*10) # Fill NaN with a large number

            # Investment Type Classification (Keep existing)
            customer_stats['投資家タイプ'] = 'その他'
            # Ensure quantile calculation has enough data, fallback to 0 if not
            q_過去取引回数 = customer_stats['過去取引回数'].quantile(0.7) if len(customer_stats) > 0 else 0

            active_mask = (
                (customer_stats['過去取引回数'] > q_過去取引回数) &
                (customer_stats['投資成功体験'] == 1)
            )
            customer_stats.loc[active_mask, '投資家タイプ'] = 'アクティブ'

            careful_mask = customer_stats['リスク管理度'] > 0.5
            customer_stats.loc[careful_mask, '投資家タイプ'] = '慎重'

            digital_mask = customer_stats['デジタル利用率'] > 0.7
            customer_stats.loc[digital_mask, '投資家タイプ'] = 'デジタル'


            # Merge
            df = df.merge(customer_stats.drop(columns=['最終取引日']), left_on='顧客ID', right_index=True, how='left') # Drop date column before merge

            # Fill NaNs for all columns added by this function
            all_trading_cols = [
                '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計',
                '売却回数', '償還損益合計', '償還回数', 'デジタル利用率',
                '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益',
                '投資成功体験', '平均利益率', 'リスク管理度', '売却損益変動性',
                '償還損益変動性', '総損益変動性', '直近取引からの日数', '投資家タイプ' # Added investment type for filling
            ]
            for col in all_trading_cols:
                 if col in df.columns:
                     if col == '直近取引からの日数':
                         df[col] = df[col].fillna(365*10) # Fill recency NaNs with very large value
                     elif col == '投資家タイプ':
                          df[col] = df[col].fillna('その他')
                     else:
                         df[col] = df[col].fillna(0) # Fill other numeric with 0


            return df

        except Exception as e:
            print(f"取引特徴量エラー (v2): {e}")
            # Fallback to dummy features if trading data processing fails
            print("Falling back to dummy trading features.")
            return add_dummy_trading_features(df) # Use the specific dummy function

    # Re-define add_real_product_features with new features
    def add_real_product_features_v2(df):
        """実際の商品データから特徴量作成 (v2 - 新機能追加)"""
        try:
            trading_with_product = trading_df.merge(
                product_df[['商品名', '商品カテゴリ', 'リスクカテゴリ', '通貨', '勧誘留意商品', '償還日']], # Added 償還日
                on='商品名', how='left'
            )

            # Map Risk Category to numerical for aggregation
            risk_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
            trading_with_product['リスクカテゴリ_数値'] = trading_with_product['リスクカテゴリ'].map(risk_mapping).fillna(2)

            # Convert 償還日 to datetime, handle errors
            trading_with_product['償還日'] = pd.to_datetime(trading_with_product['償還日'], errors='coerce')

            # Aggregate product info per customer
            customer_products = trading_with_product.groupby('顧客ID').agg({
                '商品カテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'その他',
                'リスクカテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Medium',
                'リスクカテゴリ_数値': 'mean', # New: Average numerical risk category
                '通貨': lambda x: (x == 'JPY').mean(),
                '勧誘留意商品': 'mean',
                '商品名': 'nunique', # New: Number of unique products traded (similar to 商品多様性)
                '償還日': 'max' # Get the latest maturity date among traded products
            })

            customer_products.columns = [
                '主要商品カテゴリ', '主要リスクカテゴリ', '平均リスクカテゴリ_数値', # New name
                '円建て比率', '留意商品比率', '保有商品数_ユニーク', # New name
                '最終償還日' # Use the date itself to calculate recency relative to observation date
            ]

            customer_products['リスク指向度'] = customer_products['主要リスクカテゴリ'].map(risk_mapping).fillna(2)

            # New: Days to latest maturity relative to a fixed date (e.g., train data end eve)
            analysis_date_for_aggregation = pd.to_datetime('2021-11-30') # Consistent fixed date
            customer_products['最終償還日までの日数'] = (customer_products['最終償還日'] - analysis_date_for_aggregation).dt.days.fillna(365*10) # Fill NaN with large number

            # Merge
            df = df.merge(customer_products.drop(columns=['最終償還日']), left_on='顧客ID', right_index=True, how='left') # Drop date column before merge

            # Fill NaNs for all columns added by this function
            all_product_cols = [
                 '主要商品カテゴリ', '主要リスクカテゴリ', '円建て比率', '留意商品比率',
                 'リスク指向度', '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数'
            ]

            for col in all_product_cols:
                 if col in df.columns:
                    if col in ['主要商品カテゴリ', '主要リスクカテゴリ']:
                        df[col] = df[col].fillna('その他' if col == '主要商品カテゴリ' else 'Medium')
                    elif col == '最終償還日までの日数':
                         df[col] = df[col].fillna(365*10) # Fill NaN with very large number
                    else:
                         df[col] = df[col].fillna(0.0) # Fill others with 0.0

            return df

        except Exception as e:
            print(f"商品特徴量エラー (v2): {e}")
            # Fallback to dummy features
            print("Falling back to dummy product features.")
            return add_dummy_product_features(df) # Use the specific dummy function


    # Re-define create_complete_features to use adjusted dummy functions and v2 helpers
    def create_complete_features_v2(df):
        """完全版特徴量作成 (v2 - 調整済みダミー関数使用 & v2ヘルパー)"""
        df = df.copy()

        # 基本特徴量
        df['資産規模'] = df['取得価額'] + df['時価価額']
        # Handle division by zero for 投資効率 and 年齢調整資産
        df['投資効率'] = np.where((df['取得価額'] + 1) != 0, df['時価価額'] / (df['取得価額'] + 1), 0)
        df['年齢調整資産'] = np.where((df['顧客年齢'] + 1) != 0, df['時価価額'] / (df['顧客年齢'] + 1), 0)

        # Handle potential infinite values in rank
        df['顧客ランク'] = df['時価価額'].rank(pct=True).fillna(0) # Fill NaN rank with 0
        df['年齢×資産規模'] = df['顧客年齢'] * df['資産規模']
        # Handle division by zero for 含み損益率
        df['含み損益率'] = np.where(df['取得価額'] != 0,
                                    (df['時価価額'] - df['取得価額']) / df['取得価額'], 0)

        if HAS_ALL_DATA:
            df = add_real_trading_features_v2(df) # Use v2
            df = add_real_product_features_v2(df) # Use v2
        else:
            # Call specific dummy functions if HAS_ALL_DATA is False
            df = add_dummy_trading_features(df)
            df = add_dummy_product_features(df)
            # Also call the adjusted general dummy function for other features
            df = add_dummy_features_adjusted(df)


        # フラグ特徴量
        if '資産規模' in df.columns:
            # Ensure there's enough data for quantile, fallback to 0 if not
            q_資産規模 = df['資産規模'].quantile(0.8) if len(df) > 0 else 0
            df['高資産フラグ'] = (df['資産規模'] > q_資産規模).astype(int)
        else:
             df['高資産フラグ'] = 0 # Add dummy if base feature is missing

        if '評価損益' in df.columns:
            df['含み損フラグ'] = (df['評価損益'] < 0).astype(int)
        else:
             df['含み損フラグ'] = 0 # Add dummy if base feature is missing

        if '顧客年齢' in df.columns:
            df['シニアフラグ'] = (df['顧客年齢'] >= 65).astype(int)
        else:
            df['シニアフラグ'] = 0 # Add dummy if base feature is missing

        if '投資経験（株式）' in df.columns:
            df['株式経験あり'] = (df['投資経験（株式）'] > 0).astype(int)
        else:
            df['株式経験あり'] = 0 # Add dummy if base feature is missing


        return df

    # --- Main create_advanced_features logic ---
    df = create_complete_features_v2(df) # Start with v2 complete features

    # 【新規】高度な相互作用特徴量
    if '投資効率' in df.columns and '顧客年齢' in df.columns:
         df['投資効率×年齢'] = df['投資効率'] * df['顧客年齢']
    else:
         df['投資効率×年齢'] = 0

    if 'リスク指向度' in df.columns and '投資経験（株式）' in df.columns:
         df['リスク×経験'] = df['リスク指向度'] * df['投資経験（株式）']
    else:
         df['リスク×経験'] = 0

    if '資産規模' in df.columns and '過去取引回数' in df.columns:
         df['資産×取引頻度'] = df['資産規模'] * df['過去取引回数']
    else:
         df['資産×取引頻度'] = 0

    # 【新規】比率特徴量
    if '投資成功体験' in df.columns and '過去取引回数' in df.columns:
         df['成功率'] = np.where((df['過去取引回数'] + 1) != 0, df['投資成功体験'] / (df['過去取引回数'] + 1), 0)
    else:
         df['成功率'] = 0

    if '投資効率' in df.columns:
         df['効率ランク'] = df['投資効率'].rank(pct=True).fillna(0) # Fill NaN rank with 0
    else:
         df['効率ランク'] = 0

    if '顧客年齢' in df.columns and '時価価額' in df.columns:
         # Ensure '顧客年齢' is handled correctly for grouping (convert to int if float)
         df['年齢内資産順位'] = df.groupby(df['顧客年齢'].astype(int))['時価価額'].rank(pct=True).fillna(0)
         # Handle cases where a age group has only one member -> rank is NaN
         df['年齢内資産順位'] = df['年齢内資産順位'].fillna(df['年齢内資産順位'].mean()) # Fill remaining NaNs with mean
         df['年齢内資産順位'] = df['年齢内資産順位'].fillna(0) # Fallback just in case mean is NaN

    else:
         df['年齢内資産順位'] = 0


    # 【新規】複合スコア
    # Ensure all components exist
    if all(col in df.columns for col in ['顧客ランク', '効率ランク', '投資成功体験', '過去取引回数']):
         # Ensure past_trading_count is treated as numeric, handle potential NaNs before clip
         df['総合投資力'] = (
            df['顧客ランク'].fillna(0) * 0.3 +
            df['効率ランク'].fillna(0) * 0.3 +
            df['投資成功体験'].fillna(0) * 0.2 +
            (df['過去取引回数'].fillna(0) / 50).clip(0, 1) * 0.2
         )
    else:
         df['総合投資力'] = 0


    # 【新規】異常値フラグ
    if '時価価額' in df.columns:
        # Ensure enough data for quantile, fallback to a high value or 0 if not
        q_時価価額 = df['時価価額'].quantile(0.99) if len(df) > 100 else df['時価価額'].max() + 1 # Use max if not enough data
        df['異常高資産'] = (df['時価価額'] > q_時価価額).astype(int)
    else:
        df['異常高資産'] = 0

    if '過去取引回数' in df.columns:
        # Ensure enough data for quantile, fallback to a high value or 0 if not
        q_過去取引回数 = df['過去取引回数'].quantile(0.95) if len(df) > 100 else df['過去取引回数'].max() + 1 # Use max if not enough data
        df['超アクティブ'] = (df['過去取引回数'] > q_過去取引回数).astype(int)
    else:
         df['超アクティブ'] = 0

    # Interaction features involving recreated columns (損益状況, 年齢層)
    # Recreate 損益状況 and 年齢層 within this function if they are needed for interactions
    # and might not be present if create_complete_features_v2 used dummy data.
    if '評価損益' in df.columns and '損益状況' not in df.columns:
        df['損益状況'] = 'その他'
        df.loc[df['評価損益'] < -50000, '損益状況'] = '大幅損失'
        df.loc[(df['評価損益'] >= -50000) & (df['評価損益'] < 0), '損益状況'] = '軽微損失'
        df.loc[(df['評価損益'] >= 0) & (df['評価損益'] < 50000), '損益状況'] = '軽微利益'
        df.loc[df['評価損益'] >= 50000, '損益状況'] = '大幅利益'
        # Handle potential NaNs in 評価損益
        df['損益状況'] = df['損益状況'].fillna('その他')


    if '顧客年齢' in df.columns and '年齢層' not in df.columns:
        df['年齢層'] = pd.cut(df['顧客年齢'],
                           bins=[0, 40, 50, 60, 70, 100],
                           labels=['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~'])
        # Fill NaN Age_group if any (e.g., age outside bins)
        df['年齢層'] = df['年齢層'].cat.add_categories('不明').fillna('不明')


    if '投資経験（株式）' in df.columns and '損益状況' in df.columns:
        df['投資経験_損益_大幅損失'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '大幅損失')).astype(int)
        df['投資経験_損益_軽微損失'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '軽微損失')).astype(int)
        df['投資経験_損益_軽微利益'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '軽微利益')).astype(int)
        df['投資経験_損益_大幅利益'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '大幅利益')).astype(int)
    else: # Add dummies if source columns are missing
        df['投資経験_損益_大幅損失'] = 0
        df['投資経験_損益_軽微損失'] = 0
        df['投資経験_損益_軽微利益'] = 0
        df['投資経験_損益_大幅利益'] = 0


    if '年齢層' in df.columns and '損益状況' in df.columns:
         # Use all possible categories and profit/loss statuses for robustness
         age_categories = ['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~', '不明']
         profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他'] # Include 'その他'

         for age_group in age_categories:
             for profit_loss in profit_loss_statuses:
                 col_name = f'年齢層_{age_group}_損益_{profit_loss}'.replace('~', 'under').replace('-', '_') # Clean names for columns
                 # Ensure the category exists in the dataframe before filtering
                 if age_group in df['年齢層'].cat.categories and profit_loss in df['損益状況'].unique():
                     df[col_name] = ((df['年齢層'] == age_group) & (df['損益状況'] == profit_loss)).astype(int)
                 else:
                     df[col_name] = 0 # Assign 0 if category or status is missing


    else: # Add dummies if source columns are missing
         age_categories = ['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~', '不明']
         profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他']
         for age_group in age_categories:
             for profit_loss in profit_loss_statuses:
                 col_name = f'年齢層_{age_group}_損益_{profit_loss}'.replace('~', 'under').replace('-', '_')
                 df[col_name] = 0


    if '投資方針' in df.columns and '投資経験（株式）' in df.columns:
        for policy in [1, 2, 3]: # Assume policies are 1, 2, 3
            col_name = f'投資方針_{policy}_投資経験あり'
            df[col_name] = ((df['投資方針'] == policy) & (df['投資経験（株式）'] > 0)).astype(int)
    else: # Add dummies if source columns are missing
        for policy in [1, 2, 3]:
             col_name = f'投資方針_{policy}_投資経験あり'
             df[col_name] = 0

    # Add new inferred flags
    if '損益状況' in df.columns:
         df['損失回復期待フラグ'] = (df['損益状況'] == '大幅損失').astype(int)
         df['利益確定後再投資フラグ'] = (df['損益状況'] == '大幅利益').astype(int)
    else: # Add dummies if source column is missing
         df['損失回復期待フラグ'] = 0
         df['利益確定後再投資フラグ'] = 0


    # Drop temporary columns if they were created within this function
    # Keep '損益状況' and '年齢層' if they are needed for subsequent processing (like encoding)
    # Let's drop them here as interactions are already created
    df = df.drop(columns=['損益状況', '年齢層'], errors='ignore')
    # '資産規模_区分' was used in analysis but not explicitly added as a feature here for dummy encoding
    # If needed, it should be created and then handled by the get_dummies step in the pipeline.
    # Let's add it back if we want to use it as a categorical feature.
    if '時価価額' in df.columns and '資産規模_区分' not in df.columns:
         df['資産規模_区分'] = pd.cut(df['時価価額'],
                                 bins=[0, 100000, 500000, 1000000, 5000000, float('inf')],
                                 labels=['~10万', '10-50万', '50-100万', '100-500万', '500万~'],
                                 right=False)
         df['資産規模_区分'] = df['資産規模_区分'].cat.add_categories('不明').fillna('不明')


    return df

# Modify the main pipeline to use create_advanced_features
def final_pipeline_v2(train_df, test_df):
    """最終調整パイプライン (v2 - 拡張特徴量使用)"""
    print("🚀 最終調整パイプライン（v2 - 拡張特徴量使用）")
    print("="*60)

    # 最終特徴量作成
    train_advanced = create_advanced_features(train_df.copy())
    test_advanced = create_advanced_features(test_df.copy())

    # Define the feature set to use for modeling - Include all new features
    # Let's be explicit about which columns to use as features.
    # Start with a base set and add the newly created ones.

    base_features = [
        '時価価額', '顧客ランク', '年齢×資産規模', '過去取引回数',
        '資産規模', '投資効率', '年齢調整資産', '含み損益率',
        '商品多様性', 'デジタル利用率', '投資成功体験', '過去累積損益',
        'リスク管理度', 'リスク指向度', '円建て比率', '留意商品比率',
        '顧客年齢', '投資経験（株式）', 'year', 'month', '投資方針',
        '高資産フラグ', '含み損フラグ', 'シニアフラグ', '株式経験あり'
    ]

    advanced_features_list = [
        '投資効率×年齢', 'リスク×経験', '資産×取引頻度',
        '成功率', '効率ランク', '年齢内資産順位', '総合投資力',
        '異常高資産', '超アクティブ',
        '売却損益変動性', '償還損益変動性', '総損益変動性', '直近取引からの日数',
        '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数',
        '投資経験_損益_大幅損失', '投資経験_損益_軽微損失', '投資経験_損益_軽微利益', '投資経験_損益_大幅利益',
        '投資方針_1_投資経験あり', '投資方針_2_投資経験あり', '投資方針_3_投資経験あり',
        '損失回復期待フラグ', '利益確定後再投資フラグ'
        # Add age_group x profit_loss interactions explicitly if needed, or let get_dummies handle them if they are kept as categorical
        # Let's add the specific interaction columns created
    ]

    # Add age_group x profit_loss interaction column names
    age_categories = ['under40歳', '40_50歳', '50_60歳', '60_70歳', '70歳~', '不明'] # Use cleaned names
    profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他']
    for age_group in age_categories:
        for profit_loss in profit_loss_statuses:
            col_name = f'年齢層_{age_group}_損益_{profit_loss}'
            advanced_features_list.append(col_name)

    # Add 資産規模_区分 if we want to encode it
    categorical_to_encode_list = ['投資方針', '投資家タイプ', '主要リスクカテゴリ', '主要商品カテゴリ', '資産規模_区分'] # Added 資産規模_区分


    final_features_list = list(set(base_features + advanced_features_list)) # Combine and remove duplicates

    # Separate categorical features that need one-hot encoding
    categorical_cols_for_encoding = [col for col in categorical_to_encode_list if col in final_features_list]

    # Final numerical/binary features (excluding those to be encoded)
    numeric_features = [col for col in final_features_list if col not in categorical_cols_for_encoding]


    print(f"📊 数値/バイナリ特徴量: {len(numeric_features)}個")
    print(f"📊 エンコード対象カテゴリ特徴量: {len(categorical_cols_for_encoding)}個")
    # print(f"例 (数値/バイナリ): {numeric_features[:5]}...")
    # print(f"例 (カテゴリ): {categorical_cols_for_encoding}")


    # Term別予測
    all_predictions = pd.DataFrame()
    term_aucs = []

    for term_id in range(1, 7):
        print(f"\n{'='*40}")
        print(f"Term {term_id} 最終調整 (v2)")
        print(f"{'='*40}")

        # データ準備
        train_term = train_advanced[train_advanced[f'train_term_{term_id}'] == 1].copy() # Use advanced features df
        test_term = test_advanced[test_advanced[f'test_term_{term_id}'] == 1].copy() # Use advanced features df

        # Select features - ensure they exist in the dataframe
        X_train_numeric = train_term[[col for col in numeric_features if col in train_term.columns]].fillna(0)
        X_test_numeric = test_term[[col for col in numeric_features if col in test_term.columns]].fillna(0)

        X_train_categorical = train_term[[col for col in categorical_cols_for_encoding if col in train_term.columns]]
        X_test_categorical = test_term[[col for col in categorical_cols_for_encoding if col in test_term.columns]]


        # Categorical encoding
        if not X_train_categorical.empty:
            X_train_categorical_encoded = pd.get_dummies(X_train_categorical, columns=X_train_categorical.columns, drop_first=True, dtype=int)
            X_test_categorical_encoded = pd.get_dummies(X_test_categorical, columns=X_test_categorical.columns, drop_first=True, dtype=int)

            # Combine numeric and encoded categorical features
            X_train = pd.concat([X_train_numeric, X_train_categorical_encoded], axis=1)
            X_test = pd.concat([X_test_numeric, X_test_categorical_encoded], axis=1)

            # Align columns after encoding - crucial for consistent feature sets
            train_cols = X_train.columns
            test_cols = X_test.columns

            missing_in_test = set(train_cols) - set(test_cols)
            for col in missing_in_test:
                X_test[col] = 0

            missing_in_train = set(test_cols) - set(train_cols)
            for col in missing_in_train:
                X_train[col] = 0 # Should not happen with drop_first=True and consistent data

            X_test = X_test[train_cols] # Ensure column order is the same

        else: # No categorical columns to encode
            X_train = X_train_numeric
            X_test = X_test_numeric


        y_train = train_term['y']


        print(f"最終特徴量数 (エンコーディング後): {X_train.shape[1]}個")

        # Scaling numerical features (Optional but often good practice)
        # scaler = StandardScaler()
        # numeric_cols_to_scale = X_train_numeric.columns # Scale only the original numeric features
        # if not numeric_cols_to_scale.empty:
        #      X_train[numeric_cols_to_scale] = scaler.fit_transform(X_train[numeric_cols_to_scale])
        #      X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])
        # Note: Skipping scaling for tree-based models, but leaving comment as reminder.

        # 最終モデル学習 (Use the same ensemble structure)
        ensemble = FinalEnsemble() # Re-initialize for each term
        cv_scores = ensemble.fit(X_train, y_train, term_id)
        term_aucs.append(np.mean(list(cv_scores.values())))

        # 予測
        test_pred = ensemble.predict_proba(X_test)

        # 結果保存
        pred_df = pd.DataFrame({'ID': test_term['ID'], 'predict': test_pred})
        all_predictions = pd.concat([all_predictions, pred_df]) if not all_predictions.empty else pred_df

    # 最終結果
    avg_auc = np.mean(term_aucs)
    print(f"\n{'='*60}")
    print("🎯 最終調整結果 (v2 - 拡張特徴量使用)")
    print(f"{'='*60}")

    for i, auc in enumerate(term_aucs, 1):
        print(f"Term {i}: {auc:.4f}")

    print(f"\n📈 最終CV-AUC: {avg_auc:.4f}")
    # Adjust predicted public score based on typical train/test gap
    predicted_public_auc = avg_auc - 0.027 # Maintain the same adjustment factor
    print(f"📊 予想パブリック: {predicted_public_auc:.4f}")


    # Check against target AUCs
    if avg_auc >= 0.677:
        print("🎉 0.65達成可能性大！")
    elif avg_auc >= 0.67:
        print("✅ 0.64後半達成！")
    else:
        print("📈 特徴量/モデル調整を続ける価値あり。")


    return all_predictions, avg_auc


# Test the new function and pipeline
print("🧪 テスト実行: create_advanced_features")
# Ensure necessary dataframes (trading_df, product_df) and flag (HAS_ALL_DATA) are available
# They should be from previous successful runs.

train_advanced_test = create_advanced_features(train_df.copy())
test_advanced_test = create_advanced_features(test_df.copy())

print(f"\nTrain Advanced shape: {train_advanced_test.shape}")
print(f"Test Advanced shape: {test_advanced_test.shape}")
print("\nTrain Advanced Columns:")
display(train_advanced_test.columns.tolist())
print("\nTest Advanced Columns:")
display(test_advanced_test.columns.tolist())

# Check for new columns (relative to original train_df)
original_train_cols = train_df.columns.tolist()
new_cols = [col for col in train_advanced_test.columns if col not in original_train_cols]
print(f"\nNewly added features count: {len(new_cols)}")
print(f"Newly added features: {new_cols}")

# Check for NaNs in new columns (should be filled by the function)
print("\nChecking NaNs in new columns:")
display(train_advanced_test[new_cols].isnull().sum().sort_values(ascending=False).head())
display(test_advanced_test[new_cols].isnull().sum().sort_values(ascending=False).head())

print("\n🧪 テスト完了: create_advanced_features")


🧪 テスト実行: create_advanced_features

Train Advanced shape: (90000, 111)
Test Advanced shape: (6000, 107)

Train Advanced Columns:


['ID',
 '顧客ID',
 '住所コード',
 '顧客氏名',
 '基準年月',
 '取得価額',
 '時価価額',
 '評価損益',
 '投資方針',
 'train_term_1',
 'train_term_2',
 'train_term_3',
 'train_term_4',
 'train_term_5',
 'train_term_6',
 '翌月_購入',
 '翌月_売却',
 '翌々月_購入',
 '翌々月_売却',
 'y',
 '顧客年齢',
 '投資経験（株式）',
 'year',
 'month',
 '資産規模_区分',
 '投資効率',
 '資産規模',
 '年齢調整資産',
 '顧客ランク',
 '年齢×資産規模',
 '含み損益率',
 '過去取引回数',
 '過去総取得額',
 '過去平均取得額',
 '売却損益合計',
 '売却回数',
 '売却損益変動性',
 '償還損益合計',
 '償還回数',
 '償還損益変動性',
 'デジタル利用率',
 '商品多様性',
 'ゴール設定率',
 'ロスカット設定率',
 '過去累積損益',
 '投資成功体験',
 '平均利益率',
 'リスク管理度',
 '総損益変動性',
 '直近取引からの日数',
 '投資家タイプ',
 '主要商品カテゴリ',
 '主要リスクカテゴリ',
 '平均リスクカテゴリ_数値',
 '円建て比率',
 '留意商品比率',
 '保有商品数_ユニーク',
 'リスク指向度',
 '最終償還日までの日数',
 '高資産フラグ',
 '含み損フラグ',
 'シニアフラグ',
 '株式経験あり',
 '投資効率×年齢',
 'リスク×経験',
 '資産×取引頻度',
 '成功率',
 '効率ランク',
 '年齢内資産順位',
 '総合投資力',
 '異常高資産',
 '超アクティブ',
 '投資経験_損益_大幅損失',
 '投資経験_損益_軽微損失',
 '投資経験_損益_軽微利益',
 '投資経験_損益_大幅利益',
 '年齢層_under40歳_損益_大幅損失',
 '年齢層_under40歳_損益_軽微損失',
 '年齢層_under40歳_損益_軽微利益',
 '年齢層_under40歳_損益_大幅利益',
 '年齢層_under40歳_損益_そ


Test Advanced Columns:


['Unnamed: 0',
 'ID',
 '顧客ID',
 '住所コード',
 '顧客氏名',
 '基準年月',
 '取得価額',
 '時価価額',
 '評価損益',
 '投資方針',
 'test_term_1',
 'test_term_2',
 'test_term_3',
 'test_term_4',
 'test_term_5',
 'test_term_6',
 '顧客年齢',
 '投資経験（株式）',
 'year',
 'month',
 '資産規模',
 '投資効率',
 '年齢調整資産',
 '顧客ランク',
 '年齢×資産規模',
 '含み損益率',
 '過去取引回数',
 '過去総取得額',
 '過去平均取得額',
 '売却損益合計',
 '売却回数',
 '売却損益変動性',
 '償還損益合計',
 '償還回数',
 '償還損益変動性',
 'デジタル利用率',
 '商品多様性',
 'ゴール設定率',
 'ロスカット設定率',
 '過去累積損益',
 '投資成功体験',
 '平均利益率',
 'リスク管理度',
 '総損益変動性',
 '直近取引からの日数',
 '投資家タイプ',
 '主要商品カテゴリ',
 '主要リスクカテゴリ',
 '平均リスクカテゴリ_数値',
 '円建て比率',
 '留意商品比率',
 '保有商品数_ユニーク',
 'リスク指向度',
 '最終償還日までの日数',
 '高資産フラグ',
 '含み損フラグ',
 'シニアフラグ',
 '株式経験あり',
 '投資効率×年齢',
 'リスク×経験',
 '資産×取引頻度',
 '成功率',
 '効率ランク',
 '年齢内資産順位',
 '総合投資力',
 '異常高資産',
 '超アクティブ',
 '投資経験_損益_大幅損失',
 '投資経験_損益_軽微損失',
 '投資経験_損益_軽微利益',
 '投資経験_損益_大幅利益',
 '年齢層_under40歳_損益_大幅損失',
 '年齢層_under40歳_損益_軽微損失',
 '年齢層_under40歳_損益_軽微利益',
 '年齢層_under40歳_損益_大幅利益',
 '年齢層_under40歳_損益_その他',
 '年齢層_40_50歳_損益_大幅損失',
 '年齢層_40_50歳_損益_軽微損失',



Newly added features count: 85
Newly added features: ['資産規模', '年齢調整資産', '顧客ランク', '年齢×資産規模', '含み損益率', '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計', '売却回数', '売却損益変動性', '償還損益合計', '償還回数', '償還損益変動性', 'デジタル利用率', '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益', '投資成功体験', '平均利益率', 'リスク管理度', '総損益変動性', '直近取引からの日数', '投資家タイプ', '主要商品カテゴリ', '主要リスクカテゴリ', '平均リスクカテゴリ_数値', '円建て比率', '留意商品比率', '保有商品数_ユニーク', 'リスク指向度', '最終償還日までの日数', '高資産フラグ', '含み損フラグ', 'シニアフラグ', '株式経験あり', '投資効率×年齢', 'リスク×経験', '資産×取引頻度', '成功率', '効率ランク', '年齢内資産順位', '総合投資力', '異常高資産', '超アクティブ', '投資経験_損益_大幅損失', '投資経験_損益_軽微損失', '投資経験_損益_軽微利益', '投資経験_損益_大幅利益', '年齢層_under40歳_損益_大幅損失', '年齢層_under40歳_損益_軽微損失', '年齢層_under40歳_損益_軽微利益', '年齢層_under40歳_損益_大幅利益', '年齢層_under40歳_損益_その他', '年齢層_40_50歳_損益_大幅損失', '年齢層_40_50歳_損益_軽微損失', '年齢層_40_50歳_損益_軽微利益', '年齢層_40_50歳_損益_大幅利益', '年齢層_40_50歳_損益_その他', '年齢層_50_60歳_損益_大幅損失', '年齢層_50_60歳_損益_軽微損失', '年齢層_50_60歳_損益_軽微利益', '年齢層_50_60歳_損益_大幅利益', '年齢層_50_60歳_損益_その他', '年齢層_60_70歳_損益_大幅損失', '年齢層_60_70歳_損益_軽微損失', '年齢層_60_70歳_損益_軽微利益', '年齢

,0
資産規模,0
年齢調整資産,0
顧客ランク,0
年齢×資産規模,0
含み損益率,0


,0
資産規模,0
年齢調整資産,0
顧客ランク,0
年齢×資産規模,0
含み損益率,0



🧪 テスト完了: create_advanced_features


In [39]:
print("--- 分析結果の詳細な日本語での解説 ---")

print("\n1. 資産規模別購入率:")
display(asset_purchase_rate)
print("【解説】この結果を見ると、「~10万」（10万円未満）の資産規模を持つ顧客層の購入率が約19.89%と表示されています。しかし、他の資産規模区分（10-50万、50-100万、100-500万、500万~）では顧客数が0となっており、購入率も計算できていません。これは、データ全体の資産規模の分布に対して、設定した区切りが適切ではない可能性、あるいはデータに偏りがある可能性を示唆しています。もしデータが正しければ、比較的資産規模の小さい顧客層が購入しやすい傾向があるという解釈もできますが、他の層のデータがないため、この分析結果だけで結論を出すのは難しいです。より正確な分析のためには、資産規模の区分方法を見直すか、データの特性を再確認する必要があります。")

print("\n2. 投資経験別購入率:")
display(experience_purchase.head(10))
print("【解説】株式投資の経験がある顧客層（投資経験（株式）が0より大きい）の購入率が約23.15%であるのに対し、経験がない顧客層の購入率は約18.56%となっています。この差は約4.6ポイントであり、統計的に有意な差であると考えられます。これは、投資経験がある顧客の方が、新しい商品や投資機会に対して積極的、あるいは抵抗感が少ない可能性を示しています。投資経験は、購入確率を予測する上で重要な特徴量であると言えます。")

print("\n3. 損益状況別購入率:")
display(profit_loss_purchase)
print("【解説】顧客の現在の評価損益に基づく購入率を見ると、「大幅損失」（27.16%）、「軽微損失」（25.31%）、および「大幅利益」（25.83%）の顧客層が、「軽微利益」（18.83%）の顧客層よりも高い購入率を示しています。これは非常に興味深い結果です。一般的に、損失を抱えている顧客は投資に消極的になりそうですが、このデータでは損失を取り戻そうとして（損切り覚悟やナンピンなど）、あるいは新しい機会に賭けて購入する行動が見られる可能性があります。また、大幅な利益が出ている顧客も、利益確定後の再投資や、リスクを取る余裕があるために購入しやすいと考えられます。「軽微利益」の層で購入率が低いのは、現状維持を好む傾向があるのかもしれません。")

print("\n4. 投資方針×年齢層の組み合わせ:")
display(combination_analysis)
print("【解説】投資方針と年齢層を組み合わせた分析では、いくつかの明確な傾向が見られます。まず、全体的に若年層（~40歳、40-50歳）は、他の年齢層と比較して購入率が高い傾向があります。特に「投資方針3」を選択している顧客は、多くの年齢層で最も高い購入率を示しています。注目すべきは50-60歳層で、この層はどの投資方針を選んでいても購入率が他の年齢層と比較して顕著に低いことがわかります。これは、この年齢層に特有のライフステージ（例: リタイアメント準備、教育資金のピークなど）やリスク回避傾向が影響している可能性があります。")

print("\n5. 時期別購入率:")
display(term_df)
print("【解説】分析対象期間（Term 1からTerm 6）ごとの購入率を見ると、約19.7%から19.9%の間で比較的安定しており、特定の時期に購入率が大きく変動する傾向は見られません。これは、全体的な購入行動が特定の時期イベント（例: 年末、ボーナス時期など）に強く影響されるというよりは、個別の顧客状況や市場動向に左右される側面が強い可能性を示唆しています。ただし、より細かい時間軸（例: 月ごと、週ごと）での分析を行えば、別の傾向が見える可能性もあります。")

print("\n6. 投資効率レンジ別の詳細分析:")
display(efficiency_analysis)
print("【解説】現在の投資効率（時価価額/取得価額）に基づく購入率も、「損益状況別購入率」と同様の傾向を示しています。「大幅損失」（効率0.8未満、購入率27.16%）、「横ばい」（効率0.95-1.05、購入率25.78%）、および「大幅利益」（効率1.2超、購入率25.83%）の顧客層が高い購入率を示しています。これは、評価損益だけでなく、投資効率という相対的な指標で見ても、極端な状況にある顧客や、ちょうど損益分岐点あたりにいる顧客が、購入行動を起こしやすいことを裏付けています。")

print("\n--- 全体的な結論 ---")
print("これらの分析結果から、購入確率に影響を与える主要な要因として以下が考えられます。")
print("- **投資経験**: 経験者は購入意欲が高い傾向がある。（特に株式経験）")
print("- **現在のポートフォリオ状況**: 大幅な含み損・含み益、あるいは損益がほぼゼロの顧客層が購入しやすい。これは、損失回復期待や、利益確定後の再投資意欲、あるいは現状維持からの脱却意欲など、心理的な要因が影響している可能性がある。")
print("- **年齢層と投資方針**: 若年層（特に40代以下）や「投資方針3」の顧客は購入率が高い。一方で50-60歳層は購入率が低い傾向がある。")
print("- **資産規模**: 分析には課題が残るが、データによっては小規模資産層も購入しやすい可能性が示唆された。")
print("- **時期**: 大局的には時期による大きな変動は見られない。")
print("\nこれらの知見は、単なるデモグラフィック情報だけでなく、顧客の過去の投資行動、現在のポートフォリオパフォーマンス、リスク許容度、そして心理状態などが、次の投資行動に大きく影響していることを示唆しています。これらの要素を捉える新しい特徴量が、モデルの精度向上に繋がると期待できます。")

--- 分析結果の詳細な日本語での解説 ---

1. 資産規模別購入率:


,資産規模_区分,顧客数,購入者数,購入率
0,~10万,90000,17899,19.887778
1,10-50万,0,0,NaN
2,50-100万,0,0,NaN
3,100-500万,0,0,NaN
4,500万~,0,0,NaN


【解説】この結果を見ると、「~10万」（10万円未満）の資産規模を持つ顧客層の購入率が約19.89%と表示されています。しかし、他の資産規模区分（10-50万、50-100万、100-500万、500万~）では顧客数が0となっており、購入率も計算できていません。これは、データ全体の資産規模の分布に対して、設定した区切りが適切ではない可能性、あるいはデータに偏りがある可能性を示唆しています。もしデータが正しければ、比較的資産規模の小さい顧客層が購入しやすい傾向があるという解釈もできますが、他の層のデータがないため、この分析結果だけで結論を出すのは難しいです。より正確な分析のためには、資産規模の区分方法を見直すか、データの特性を再確認する必要があります。

2. 投資経験別購入率:


,投資経験（株式）,顧客数,購入率
0,0.0,63990,18.560713
1,1.0,26010,23.152634


【解説】株式投資の経験がある顧客層（投資経験（株式）が0より大きい）の購入率が約23.15%であるのに対し、経験がない顧客層の購入率は約18.56%となっています。この差は約4.6ポイントであり、統計的に有意な差であると考えられます。これは、投資経験がある顧客の方が、新しい商品や投資機会に対して積極的、あるいは抵抗感が少ない可能性を示しています。投資経験は、購入確率を予測する上で重要な特徴量であると言えます。

3. 損益状況別購入率:


...

【解説】投資方針と年齢層を組み合わせた分析では、いくつかの明確な傾向が見られます。まず、全体的に若年層（~40歳、40-50歳）は、他の年齢層と比較して購入率が高い傾向があります。特に「投資方針3」を選択している顧客は、多くの年齢層で最も高い購入率を示しています。注目すべきは50-60歳層で、この層はどの投資方針を選んでいても購入率が他の年齢層と比較して顕著に低いことがわかります。これは、この年齢層に特有のライフステージ（例: リタイアメント準備、教育資金のピークなど）やリスク回避傾向が影響している可能性があります。

5. 時期別購入率:


,Term,顧客数,購入率
0,1,80000,19.722500
1,2,82000,19.760976
2,3,84000,19.803571
3,4,86000,19.845349
4,5,88000,19.886364
5,6,90000,19.887778


【解説】分析対象期間（Term 1からTerm 6）ごとの購入率を見ると、約19.7%から19.9%の間で比較的安定しており、特定の時期に購入率が大きく変動する傾向は見られません。これは、全体的な購入行動が特定の時期イベント（例: 年末、ボーナス時期など）に強く影響されるというよりは、個別の顧客状況や市場動向に左右される側面が強い可能性を示唆しています。ただし、より細かい時間軸（例: 月ごと、週ごと）での分析を行えば、別の傾向が見える可能性もあります。

6. 投資効率レンジ別の詳細分析:


,投資効率レンジ,顧客数,購入者数,購入率
0,大幅損失,394,107,27.157360
1,軽微損失,5103,1129,22.124241
2,横ばい,38566,9942,25.779184
3,軽微利益,16752,4269,25.483524
4,大幅利益,9285,2398,25.826602


【解説】現在の投資効率（時価価額/取得価額）に基づく購入率も、「損益状況別購入率」と同様の傾向を示しています。「大幅損失」（効率0.8未満、購入率27.16%）、「横ばい」（効率0.95-1.05、購入率25.78%）、および「大幅利益」（効率1.2超、購入率25.83%）の顧客層が高い購入率を示しています。これは、評価損益だけでなく、投資効率という相対的な指標で見ても、極端な状況にある顧客や、ちょうど損益分岐点あたりにいる顧客が、購入行動を起こしやすいことを裏付けています。

--- 全体的な結論 ---
これらの分析結果から、購入確率に影響を与える主要な要因として以下が考えられます。
- **投資経験**: 経験者は購入意欲が高い傾向がある。（特に株式経験）
- **現在のポートフォリオ状況**: 大幅な含み損・含み益、あるいは損益がほぼゼロの顧客層が購入しやすい。これは、損失回復期待や、利益確定後の再投資意欲、あるいは現状維持からの脱却意欲など、心理的な要因が影響している可能性がある。
- **年齢層と投資方針**: 若年層（特に40代以下）や「投資方針3」の顧客は購入率が高い。一方で50-60歳層は購入率が低い傾向がある。
- **資産規模**: 分析には課題が残るが、データによっては小規模資産層も購入しやすい可能性が示唆された。
- **時期**: 大局的には時期による大きな変動は見られない。

これらの知見は、単なるデモグラフィック情報だけでなく、顧客の過去の投資行動、現在のポートフォリオパフォーマンス、リスク許容度、そして心理状態などが、次の投資行動に大きく影響していることを示唆しています。これらの要素を捉える新しい特徴量が、モデルの精度向上に繋がると期待できます。


In [40]:
print("\n📊 住所コード別購入率:")
# Check if 住所コード exists and is not all nulls before grouping
if '住所コード' in train_df.columns and not train_df['住所コード'].isnull().all():
    address_purchase_rate = train_df.groupby('住所コード')['y'].agg(['count', 'sum', 'mean']).reset_index()
    address_purchase_rate.columns = ['住所コード', '顧客数', '購入者数', '購入率']
    address_purchase_rate['購入率'] = address_purchase_rate['購入率'] * 100
    # Sort by purchase rate to easily see codes with high/low rates
    address_purchase_rate = address_purchase_rate.sort_values('購入率', ascending=False)
    display(address_purchase_rate)
    print("【解説】住所コードごとの顧客数、購入者数、購入率を示しています。購入率が高い/低い住所コードがあるか確認してください。")
else:
    print("「住所コード」カラムが見つからないか、データがありません。")


📊 住所コード別購入率:


,住所コード,顧客数,購入者数,購入率
3,A-4,900,275,30.555556
27,G-4,900,236,26.222222
34,I-3,2700,705,26.111111
11,C-4,900,220,24.444444
25,G-2,2700,635,23.518519
24,G-1,2700,626,23.185185
13,D-2,2700,624,23.111111
23,F-4,900,208,23.111111
33,I-2,2700,604,22.370370
6,B-3,2700,598,22.148148


【解説】住所コードごとの顧客数、購入者数、購入率を示しています。購入率が高い/低い住所コードがあるか確認してください。


In [41]:
# 1. Identify key trends from previous analysis:
#    - Investment experience matters.
#    - Profit/Loss and Investment Efficiency extremes (loss or high gain/break-even) matter.
#    - Age group (younger generally higher, 50-60 lower) and Investment Policy (Policy 3 higher) matter.
#    - Asset size analysis was inconclusive and needs better handling.
#    - Term doesn't seem to have a strong impact alone.
#    - Address code shows variation in purchase rates.

# 2. Brainstorm new features based on these trends and available data (trading_df, product_df):

new_feature_ideas = [
    # Features from Trading Data (more detailed behavior)
    {"name": "過去取引頻度_年", "description": "過去の総取引回数を年数で割ったもの。アクティブな顧客ほど高い。", "source": "trading_df"},
    {"name": "直近取引からの日数", "description": "最後の取引日からの経過日数。最近取引した顧客は再度購入しやすいか。", "source": "trading_df"},
    {"name": "特定カテゴリ投資比率", "description": "特定の高リスク/高リターンカテゴリ（例: 外国株式、高利回り債券）への投資額比率。リスク志向をより詳細に捉える。", "source": "trading_df, product_df"},
    {"name": "損益額変動性", "description": "過去の売却・償還損益の標準偏差。損益のブレが大きい顧客はリスク許容度が高いか。", "source": "trading_df"},
    {"name": "平均保有期間", "description": "各商品の取得日から売却/償還日までの平均期間。短期志向か長期志向か。", "source": "trading_df"}, # Requires linking trades of the same product
    {"name": "チャネル別取引比率", "description": "オンライン取引と対面取引の比率。デジタル利用度をより詳細に。", "source": "trading_df"},

    # Features from Product Data (current portfolio characteristics)
    {"name": "保有商品数_カテゴリ別", "description": "保有している商品カテゴリの数。商品多様性をより詳細に。", "source": "trading_df, product_df"}, # Re-evaluating existing idea with more detail
    {"name": "保有商品_平均リスクカテゴリ", "description": "保有商品のリスクカテゴリの平均値（数値化後）。ポートフォリオ全体のリスクレベル。", "source": "trading_df, product_df"},
    {"name": "保有商品_平均償還日までの期間", "description": "保有商品の償還日までの平均期間（あれば）。長期保有志向か。", "source": "trading_df, product_df"},

    # Interaction and Transformation Features (combining existing info)
    {"name": "顧客年齢_二乗", "description": "顧客年齢の非線形効果を捉える。", "source": "train_df, test_df (transformed)"},
    {"name": "時価価額_対数", "description": "資産規模の歪んだ分布を正規化し、影響を滑らかにする。", "source": "train_df, test_df (transformed)"},
    {"name": "投資経験×損益状況", "description": "投資経験と損益状況の組み合わせ効果。経験者は損失をどう受け止めるか。", "source": "train_df, test_df (interaction)"},
    {"name": "年齢層×損益状況", "description": "年齢層と損益状況の組み合わせ効果。", "source": "train_df, test_df (interaction)"},
    {"name": "投資方針×投資経験", "description": "投資方針と投資経験の組み合わせ効果。", "source": "train_df, test_df (interaction)"},
    {"name": "資産規模_区分_ダミー", "description": "資産規模区分をダミー変数化。分析で示された違いをモデルに学習させる。", "source": "train_df, test_df (dummy)"},
    {"name": "住所コード_ダミー", "description": "住所コードをダミー変数化。地域ごとの購入率の違いをモデルに学習させる。", "source": "train_df, test_df (dummy)"}, # New idea based on recent analysis

    # Behavioral/Inferred Features
    {"name": "損失回復期待フラグ", "description": "大幅損失顧客が、損失を取り戻そうとして購入する傾向があるか。", "source": "train_df, test_df (inferred from 損益状況)"},
    {"name": "利益確定後再投資フラグ", "description": "大幅利益顧客が、利益を確定して別の商品に再投資する傾向があるか。", "source": "train_df, test_df (inferred from 損益状況)"},
]

# 3. Print the list of ideas
print("### 新しい特徴量のアイデア ###")
for i, idea in enumerate(new_feature_ideas):
    print(f"{i+1}. {idea['name']}: {idea['description']} (データソース: {idea['source']})")

print("\nこれらの特徴量は、過去の分析で示された顧客の行動、ポートフォリオ特性、既存特徴量の複雑な関係性を捉えることで、モデルの予測力向上に貢献する可能性があります。特に、過去の取引詳細や商品情報を活用した特徴量は、顧客一人ひとりの投資プロファイルをより深く理解するために有効です。")

### 新しい特徴量のアイデア ###
1. 過去取引頻度_年: 過去の総取引回数を年数で割ったもの。アクティブな顧客ほど高い。 (データソース: trading_df)
2. 直近取引からの日数: 最後の取引日からの経過日数。最近取引した顧客は再度購入しやすいか。 (データソース: trading_df)
3. 特定カテゴリ投資比率: 特定の高リスク/高リターンカテゴリ（例: 外国株式、高利回り債券）への投資額比率。リスク志向をより詳細に捉える。 (データソース: trading_df, product_df)
4. 損益額変動性: 過去の売却・償還損益の標準偏差。損益のブレが大きい顧客はリスク許容度が高いか。 (データソース: trading_df)
5. 平均保有期間: 各商品の取得日から売却/償還日までの平均期間。短期志向か長期志向か。 (データソース: trading_df)
6. チャネル別取引比率: オンライン取引と対面取引の比率。デジタル利用度をより詳細に。 (データソース: trading_df)
7. 保有商品数_カテゴリ別: 保有している商品カテゴリの数。商品多様性をより詳細に。 (データソース: trading_df, product_df)
8. 保有商品_平均リスクカテゴリ: 保有商品のリスクカテゴリの平均値（数値化後）。ポートフォリオ全体のリスクレベル。 (データソース: trading_df, product_df)
9. 保有商品_平均償還日までの期間: 保有商品の償還日までの平均期間（あれば）。長期保有志向か。 (データソース: trading_df, product_df)
10. 顧客年齢_二乗: 顧客年齢の非線形効果を捉える。 (データソース: train_df, test_df (transformed))
11. 時価価額_対数: 資産規模の歪んだ分布を正規化し、影響を滑らかにする。 (データソース: train_df, test_df (transformed))
12. 投資経験×損益状況: 投資経験と損益状況の組み合わせ効果。経験者は損失をどう受け止めるか。 (データソース: train_df, test_df (interaction))
13. 年齢層×損益状況: 年齢層と損益状況の組み合わせ効果。 (データソ

In [ ]:
# Copy and rename the existing function
def create_advanced_features(df):
    """拡張版特徴量作成（新しいアイデアを含む）"""
    df = df.copy()

    # --- Helper Functions (Nested to keep them local to this feature creation process) ---
    # Re-define add_dummy_trading_features
    def add_dummy_trading_features(df):
        """ダミー取引特徴量追加"""
        dummy_trading_cols = [
            '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計',
            '売却回数', '償還損益合計', '償還回数', 'デジタル利用率',
            '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益',
            '投資成功体験', '平均利益率', 'リスク管理度', '売却損益変動性',
            '償還損益変動性', '総損益変動性', '直近取引からの日数' # Add new dummy cols
        ]
        for feature in dummy_trading_cols:
             if feature not in df.columns:
                 if feature == '直近取引からの日数':
                     df[feature] = 365 * 10 # Large number for recency if no data
                 else:
                     df[feature] = 0

        if '投資家タイプ' not in df.columns:
            df['投資家タイプ'] = 'その他'

        return df

    # Re-define add_dummy_product_features
    def add_dummy_product_features(df):
         """ダミー商品特徴量追加"""
         dummy_product_cols = [
             '主要商品カテゴリ', '主要リスクカテゴリ', '円建て比率', '留意商品比率',
             'リスク指向度', '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数' # Add new dummy cols
         ]
         for feature in dummy_product_cols:
             if feature not in df.columns:
                 if feature in ['主要商品カテゴリ', '主要リスクカテゴリ']:
                     df[feature] = 'その他' if feature == '主要商品カテゴリ' else 'Medium'
                 elif feature == '最終償還日までの日数':
                     df[feature] = 365 * 10 # Large number for recency if no data
                 else:
                     df[feature] = 0.0

         return df

    # Adjust add_dummy_features to check for column existence and cover new dummies
    def add_dummy_features_adjusted(df):
        """ダミー特徴量追加 (既存カラムを上書きしない)"""
        dummy_features = [
            '過去取引回数', '過去累積損益', '投資成功体験', 'デジタル利用率',
            '商品多様性', '主要リスクカテゴリ', '投資家タイプ', 'リスク管理度', 'リスク指向度',
            # Add new dummy features here to ensure they are covered if HAS_ALL_DATA is False and specific dummy functions aren't called
            '売却損益変動性', '償還損益変動性', '総損益変動性', '直近取引からの日数',
            '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数',
            '円建て比率', '留意商品比率' # Ensure these base features are covered
        ]
        for feature in dummy_features:
            if feature not in df.columns: # Only add if column doesn't exist
                if feature in ['主要リスクカテゴリ', '投資家タイプ', '主要商品カテゴリ']: # Added 主要商品カテゴリ
                    df[feature] = 'その他' if feature == '投資家タイプ' else 'Medium' if feature == '主要リスクカテゴリ' else 'その他'
                elif feature == '直近取引からの日数' or feature == '最終償還日までの日数':
                     df[feature] = 365 * 10 # Large number for recency/maturity if no data
                elif feature in ['円建て比率', '留意商品比率', '平均リスクカテゴリ_数値', '保有商品数_ユニーク']:
                     df[feature] = 0.0
                else:
                    df[feature] = 0 # Default to 0 for other numeric dummies

        return df

    # Re-define add_real_trading_features with new features
    def add_real_trading_features_v2(df):
        """実際の約定データから特徴量作成 (v2 - 新機能追加)"""
        try:
            trading_df_copy = trading_df.copy() # Work on a copy to avoid modifying global df
            trading_df_copy['取引日'] = pd.to_datetime(trading_df_copy['取引日'])
            # Filter trading data up to a certain point relative to the analysis date if possible,
            # or use a fixed past window (e.g., up to '2021-11-30' for train data aggregation)
            # For testing train_df and test_df directly with this function outside the pipeline context,
            # we need to use a date appropriate for the data's timestamp.
            # However, the pipeline aggregates per term based on train_term_X flags.
            # Let's keep the fixed date logic for now, assuming this function is called
            # within a context that handles time slicing, or accepts a cut-off date.
            # For this general feature creation function, a fixed date (like the train data end date eve) is safer.
            analysis_date_for_aggregation = pd.to_datetime('2021-11-30') # Using a fixed date before test set

            past_trading = trading_df_copy[trading_df_copy['取引日'] <= analysis_date_for_aggregation]

            customer_stats = past_trading.groupby('顧客ID').agg({
                '取得価額': ['count', 'sum', 'mean'],
                '売却損益': ['sum', 'count', 'std'], # Added std for volatility
                '償還損益': ['sum', 'count', 'std'], # Added std for volatility
                'オンライン取引フラグ': 'mean',
                '商品名': 'nunique',
                'ゴール設定実施': 'mean',
                'ロスカット設定実施': 'mean',
                '取引日': 'max' # Added max date for recency
            }).fillna(0)

            # Flatten multi-level columns
            customer_stats.columns = ['_'.join(col).strip('_') for col in customer_stats.columns.values]

            customer_stats = customer_stats.rename(columns={
                '取得価額_count': '過去取引回数',
                '取得価額_sum': '過去総取得額',
                '取得価額_mean': '過去平均取得額',
                '売却損益_sum': '売却損益合計',
                '売却損益_count': '売却回数',
                '売却損益_std': '売却損益変動性', # New
                '償還損益_sum': '償還損益合計',
                '償還損益_count': '償還回数',
                '償還損益_std': '償還損益変動性', # New
                'オンライン取引フラグ_mean': 'デジタル利用率',
                '商品名_nunique': '商品多様性',
                'ゴール設定実施_mean': 'ゴール設定率',
                'ロスカット設定実施_mean': 'ロスカット設定率',
                '取引日_max': '最終取引日' # New
            })

            customer_stats['過去累積損益'] = customer_stats['売却損益合計'] + customer_stats['償還損益合計']
            customer_stats['投資成功体験'] = (customer_stats['過去累積損益'] > 0).astype(int)
            customer_stats['平均利益率'] = np.where((customer_stats['過去総取得額'] + 1) != 0,
                                                 customer_stats['過去累積損益'] / (customer_stats['過去総取得額'] + 1), 0)

            customer_stats['リスク管理度'] = (customer_stats['ゴール設定率'] + customer_stats['ロスカット設定率']) / 2.0 # Use 2.0 for float division

            # New: Total損益変動性 - Fill NaN std with 0 before summing/averaging
            customer_stats['売却損益変動性'] = customer_stats['売却損益変動性'].fillna(0)
            customer_stats['償還損益変動性'] = customer_stats['償還損益変動性'].fillna(0)
            customer_stats['総損益変動性'] = customer_stats[['売却損益変動性', '償還損益変動性']].mean(axis=1)


            # New: Recency (Days since last trade relative to analysis date)
            customer_stats['直近取引からの日数'] = (analysis_date_for_aggregation - customer_stats['最終取引日']).dt.days.fillna(365*10) # Fill NaN with a large number

            # New: Channel Mix (approximated by mean online flag)
            # 'デジタル利用率' already exists and captures this. Can add a binary flag if needed.
            # customer_stats['高オンライン比率フラグ'] = (customer_stats['デジタル利用率'] > 0.8).astype(int)

            # Investment Type Classification (Keep existing)
            customer_stats['投資家タイプ'] = 'その他'
            # Ensure quantile calculation has enough data, fallback to 0 if not
            q_過去取引回数 = customer_stats['過去取引回数'].quantile(0.7) if len(customer_stats) > 0 else 0

            active_mask = (
                (customer_stats['過去取引回数'] > q_過去取引回数) &
                (customer_stats['投資成功体験'] == 1)
            )
            customer_stats.loc[active_mask, '投資家タイプ'] = 'アクティブ'

            careful_mask = customer_stats['リスク管理度'] > 0.5
            customer_stats.loc[careful_mask, '投資家タイプ'] = '慎重'

            digital_mask = customer_stats['デジタル利用率'] > 0.7
            customer_stats.loc[digital_mask, '投資家タイプ'] = 'デジタル'


            # Merge
            df = df.merge(customer_stats.drop(columns=['最終取引日']), left_on='顧客ID', right_index=True, how='left') # Drop date column before merge

            # Fill NaNs for all columns added by this function
            all_trading_cols = [
                '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計',
                '売却回数', '償還損益合計', '償還回数', 'デジタル利用率',
                '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益',
                '投資成功体験', '平均利益率', 'リスク管理度', '売却損益変動性',
                '償還損益変動性', '総損益変動性', '直近取引からの日数', '投資家タイプ' # Added investment type for filling
            ]
            for col in all_trading_cols:
                 if col in df.columns:
                     if col == '直近取引からの日数':
                         df[col] = df[col].fillna(365*10) # Fill recency NaNs with very large value
                     elif col == '投資家タイプ':
                          df[col] = df[col].fillna('その他')
                     else:
                         df[col] = df[col].fillna(0) # Fill other numeric with 0


            return df

        except Exception as e:
            print(f"取引特徴量エラー (v2): {e}")
            # Fallback to dummy features if trading data processing fails
            print("Falling back to dummy trading features.")
            return add_dummy_trading_features(df) # Use the specific dummy function

    # Re-define add_real_product_features with new features
    def add_real_product_features_v2(df):
        """実際の商品データから特徴量作成 (v2 - 新機能追加)"""
        try:
            trading_with_product = trading_df.merge(
                product_df[['商品名', '商品カテゴリ', 'リスクカテゴリ', '通貨', '勧誘留意商品', '償還日']], # Added 償還日
                on='商品名', how='left'
            )

            # Map Risk Category to numerical for aggregation
            risk_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
            trading_with_product['リスクカテゴリ_数値'] = trading_with_product['リスクカテゴリ'].map(risk_mapping).fillna(2)

            # Convert 償還日 to datetime, handle errors
            trading_with_product['償還日'] = pd.to_datetime(trading_with_product['償還日'], errors='coerce')

            # Aggregate product info per customer
            customer_products = trading_with_product.groupby('顧客ID').agg({
                '商品カテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'その他',
                'リスクカテゴリ': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Medium',
                'リスクカテゴリ_数値': 'mean', # New: Average numerical risk category
                '通貨': lambda x: (x == 'JPY').mean(),
                '勧誘留意商品': 'mean',
                '商品名': 'nunique', # New: Number of unique products traded (similar to 商品多様性)
                '償還日': 'max' # Get the latest maturity date among traded products
            })

            customer_products.columns = [
                '主要商品カテゴリ', '主要リスクカテゴリ', '平均リスクカテゴリ_数値', # New name
                '円建て比率', '留意商品比率', '保有商品数_ユニーク', # New name
                '最終償還日' # Use the date itself to calculate recency relative to observation date
            ]

            customer_products['リスク指向度'] = customer_products['主要リスクカテゴリ'].map(risk_mapping).fillna(2)

            # New: Days to latest maturity relative to a fixed date (e.g., train data end eve)
            analysis_date_for_aggregation = pd.to_datetime('2021-11-30') # Consistent fixed date
            customer_products['最終償還日までの日数'] = (customer_products['最終償還日'] - analysis_date_for_aggregation).dt.days.fillna(365*10) # Fill NaN with large number

            # Merge
            df = df.merge(customer_products.drop(columns=['最終償還日']), left_on='顧客ID', right_index=True, how='left') # Drop date column before merge

            # Fill NaNs for all columns added by this function
            all_product_cols = [
                 '主要商品カテゴリ', '主要リスクカテゴリ', '円建て比率', '留意商品比率',
                 'リスク指向度', '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数'
            ]

            for col in all_product_cols:
                 if col in df.columns:
                    if col in ['主要商品カテゴリ', '主要リスクカテゴリ']:
                        df[col] = df[col].fillna('その他' if col == '主要商品カテゴリ' else 'Medium')
                    elif col == '最終償還日までの日数':
                         df[col] = df[col].fillna(365*10) # Fill NaN with very large number
                    else:
                         df[col] = df[col].fillna(0.0) # Fill others with 0.0

            return df

        except Exception as e:
            print(f"商品特徴量エラー (v2): {e}")
            # Fallback to dummy features
            print("Falling back to dummy product features.")
            return add_dummy_product_features(df) # Use the specific dummy function


    # Re-define create_complete_features to use adjusted dummy functions and v2 helpers
    def create_complete_features_v2(df):
        """完全版特徴量作成 (v2 - 調整済みダミー関数使用 & v2ヘルパー)"""
        df = df.copy()

        # 基本特徴量
        df['資産規模'] = df['取得価額'] + df['時価価額']
        # Handle division by zero for 投資効率 and 年齢調整資産
        df['投資効率'] = np.where((df['取得価額'] + 1) != 0, df['時価価額'] / (df['取得価額'] + 1), 0)
        df['年齢調整資産'] = np.where((df['顧客年齢'] + 1) != 0, df['時価価額'] / (df['顧客年齢'] + 1), 0)

        # Handle potential infinite values in rank
        df['顧客ランク'] = df['時価価額'].rank(pct=True).fillna(0) # Fill NaN rank with 0
        df['年齢×資産規模'] = df['顧客年齢'] * df['資産規模']
        # Handle division by zero for 含み損益率
        df['含み損益率'] = np.where(df['取得価額'] != 0,
                                    (df['時価価額'] - df['取得価額']) / df['取得価額'], 0)

        if HAS_ALL_DATA:
            df = add_real_trading_features_v2(df) # Use v2
            df = add_real_product_features_v2(df) # Use v2
        else:
            # Call specific dummy functions if HAS_ALL_DATA is False
            df = add_dummy_trading_features(df)
            df = add_dummy_product_features(df)
            # Also call the adjusted general dummy function for other features
            df = add_dummy_features_adjusted(df)


        # フラグ特徴量
        if '資産規模' in df.columns:
            # Ensure there's enough data for quantile, fallback to 0 if not
            q_資産規模 = df['資産規模'].quantile(0.8) if len(df) > 0 else 0
            df['高資産フラグ'] = (df['資産規模'] > q_資産規模).astype(int)
        else:
             df['高資産フラグ'] = 0 # Add dummy if base feature is missing

        if '評価損益' in df.columns:
            df['含み損フラグ'] = (df['評価損益'] < 0).astype(int)
        else:
             df['含み損フラグ'] = 0 # Add dummy if base feature is missing

        if '顧客年齢' in df.columns:
            df['シニアフラグ'] = (df['顧客年齢'] >= 65).astype(int)
        else:
            df['シニアフラグ'] = 0 # Add dummy if base feature is missing

        if '投資経験（株式）' in df.columns:
            df['株式経験あり'] = (df['投資経験（株式）'] > 0).astype(int)
        else:
            df['株式経験あり'] = 0 # Add dummy if base feature is missing


        return df

    # --- Main create_advanced_features logic ---
    df = create_complete_features_v2(df) # Start with v2 complete features

    # 【新規】高度な相互作用特徴量
    if '投資効率' in df.columns and '顧客年齢' in df.columns:
         df['投資効率×年齢'] = df['投資効率'] * df['顧客年齢']
    else:
         df['投資効率×年齢'] = 0

    if 'リスク指向度' in df.columns and '投資経験（株式）' in df.columns:
         df['リスク×経験'] = df['リスク指向度'] * df['投資経験（株式）']
    else:
         df['リスク×経験'] = 0

    if '資産規模' in df.columns and '過去取引回数' in df.columns:
         df['資産×取引頻度'] = df['資産規模'] * df['過去取引回数']
    else:
         df['資産×取引頻度'] = 0

    # 【新規】比率特徴量
    if '投資成功体験' in df.columns and '過去取引回数' in df.columns:
         df['成功率'] = np.where((df['過去取引回数'] + 1) != 0, df['投資成功体験'] / (df['過去取引回数'] + 1), 0)
    else:
         df['成功率'] = 0

    if '投資効率' in df.columns:
         df['効率ランク'] = df['投資効率'].rank(pct=True).fillna(0) # Fill NaN rank with 0
    else:
         df['効率ランク'] = 0

    if '顧客年齢' in df.columns and '時価価額' in df.columns:
         # Ensure '顧客年齢' is handled correctly for grouping (convert to int if float)
         df['年齢内資産順位'] = df.groupby(df['顧客年齢'].astype(int))['時価価額'].rank(pct=True).fillna(0)
         # Handle cases where a age group has only one member -> rank is NaN
         df['年齢内資産順位'] = df['年齢内資産順位'].fillna(df['年齢内資産順位'].mean()) # Fill remaining NaNs with mean
         df['年齢内資産順位'] = df['年齢内資産順位'].fillna(0) # Fallback just in case mean is NaN

    else:
         df['年齢内資産順位'] = 0


    # 【新規】複合スコア
    # Ensure all components exist
    if all(col in df.columns for col in ['顧客ランク', '効率ランク', '投資成功体験', '過去取引回数']):
         # Ensure past_trading_count is treated as numeric, handle potential NaNs before clip
         df['総合投資力'] = (
            df['顧客ランク'].fillna(0) * 0.3 +
            df['効率ランク'].fillna(0) * 0.3 +
            df['投資成功体験'].fillna(0) * 0.2 +
            (df['過去取引回数'].fillna(0) / 50).clip(0, 1) * 0.2
         )
    else:
         df['総合投資力'] = 0


    # 【新規】異常値フラグ
    if '時価価額' in df.columns:
        # Ensure enough data for quantile, fallback to a high value or 0 if not
        q_時価価額 = df['時価価額'].quantile(0.99) if len(df) > 100 else df['時価価額'].max() + 1 # Use max if not enough data
        df['異常高資産'] = (df['時価価額'] > q_時価価額).astype(int)
    else:
        df['異常高資産'] = 0

    if '過去取引回数' in df.columns:
        # Ensure enough data for quantile, fallback to a high value or 0 if not
        q_過去取引回数 = df['過去取引回数'].quantile(0.95) if len(df) > 100 else df['過去取引回数'].max() + 1 # Use max if not enough data
        df['超アクティブ'] = (df['過去取引回数'] > q_過去取引回数).astype(int)
    else:
         df['超アクティブ'] = 0

    # Interaction features involving recreated columns (損益状況, 年齢層)
    # Recreate 損益状況 and 年齢層 within this function if they are needed for interactions
    # and might not be present if create_complete_features_v2 used dummy data.
    if '評価損益' in df.columns and '損益状況' not in df.columns:
        df['損益状況'] = 'その他'
        df.loc[df['評価損益'] < -50000, '損益状況'] = '大幅損失'
        df.loc[(df['評価損益'] >= -50000) & (df['評価損益'] < 0), '損益状況'] = '軽微損失'
        df.loc[(df['評価損益'] >= 0) & (df['評価損益'] < 50000), '損益状況'] = '軽微利益'
        df.loc[df['評価損益'] >= 50000, '損益状況'] = '大幅利益'
        # Handle potential NaNs in 評価損益
        df['損益状況'] = df['損益状況'].fillna('その他')


    if '顧客年齢' in df.columns and '年齢層' not in df.columns:
        df['年齢層'] = pd.cut(df['顧客年齢'],
                           bins=[0, 40, 50, 60, 70, 100],
                           labels=['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~'])
        # Fill NaN Age_group if any (e.g., age outside bins)
        df['年齢層'] = df['年齢層'].cat.add_categories('不明').fillna('不明')


    if '投資経験（株式）' in df.columns and '損益状況' in df.columns:
        df['投資経験_損益_大幅損失'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '大幅損失')).astype(int)
        df['投資経験_損益_軽微損失'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '軽微損失')).astype(int)
        df['投資経験_損益_軽微利益'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '軽微利益')).astype(int)
        df['投資経験_損益_大幅利益'] = ((df['投資経験（株式）'] > 0) & (df['損益状況'] == '大幅利益')).astype(int)
    else: # Add dummies if source columns are missing
        df['投資経験_損益_大幅損失'] = 0
        df['投資経験_損益_軽微損失'] = 0
        df['投資経験_損益_軽微利益'] = 0
        df['投資経験_損益_大幅利益'] = 0


    if '年齢層' in df.columns and '損益状況' in df.columns:
         # Use all possible categories and profit/loss statuses for robustness
         age_categories = ['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~', '不明']
         profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他'] # Include 'その他'

         for age_group in age_categories:
             for profit_loss in profit_loss_statuses:
                 col_name = f'年齢層_{age_group}_損益_{profit_loss}'.replace('~', 'under').replace('-', '_') # Clean names for columns
                 # Ensure the category exists in the dataframe before filtering
                 if age_group in df['年齢層'].cat.categories and profit_loss in df['損益状況'].unique():
                     df[col_name] = ((df['年齢層'] == age_group) & (df['損益状況'] == profit_loss)).astype(int)
                 else:
                     df[col_name] = 0 # Assign 0 if category or status is missing


    else: # Add dummies if source columns are missing
         age_categories = ['~40歳', '40-50歳', '50-60歳', '60-70歳', '70歳~', '不明']
         profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他']
         for age_group in age_categories:
             for profit_loss in profit_loss_statuses:
                 col_name = f'年齢層_{age_group}_損益_{profit_loss}'.replace('~', 'under').replace('-', '_')
                 df[col_name] = 0


    if '投資方針' in df.columns and '投資経験（株式）' in df.columns:
        for policy in [1, 2, 3]: # Assume policies are 1, 2, 3
            col_name = f'投資方針_{policy}_投資経験あり'
            df[col_name] = ((df['投資方針'] == policy) & (df['投資経験（株式）'] > 0)).astype(int)
    else: # Add dummies if source columns are missing
        for policy in [1, 2, 3]:
             col_name = f'投資方針_{policy}_投資経験あり'
             df[col_name] = 0

    # Add new inferred flags
    if '損益状況' in df.columns:
         df['損失回復期待フラグ'] = (df['損益状況'] == '大幅損失').astype(int)
         df['利益確定後再投資フラグ'] = (df['損益状況'] == '大幅利益').astype(int)
    else: # Add dummies if source column is missing
         df['損失回復期待フラグ'] = 0
         df['利益確定後再投資フラグ'] = 0


    # Drop temporary columns if they were created within this function
    # Keep '損益状況' and '年齢層' if they are needed for subsequent processing (like encoding)
    # Let's drop them here as interactions are already created
    df = df.drop(columns=['損益状況', '年齢層'], errors='ignore')
    # '資産規模_区分' was used in analysis but not explicitly added as a feature here for dummy encoding
    # If needed, it should be created and then handled by the get_dummies step in the pipeline.
    # Let's add it back if we want to use it as a categorical feature.
    if '時価価額' in df.columns and '資産規模_区分' not in df.columns:
         df['資産規模_区分'] = pd.cut(df['時価価額'],
                                 bins=[0, 100000, 500000, 1000000, 5000000, float('inf')],
                                 labels=['~10万', '10-50万', '50-100万', '100-500万', '500万~'],
                                 right=False)
         df['資産規模_区分'] = df['資産規模_区分'].cat.add_categories('不明').fillna('不明')

    # Add 住所コード as a categorical feature for later encoding
    if '住所コード' in df.columns:
        df['住所コード'] = df['住所コード'].astype('category')
        # Add '不明' category and fill NaNs if any
        if '不明' not in df['住所コード'].cat.categories:
             df['住所コード'] = df['住所コード'].cat.add_categories('不明')
        df['住所コード'] = df['住所コード'].fillna('不明')
    else:
        # Add a dummy 住所コード column if missing
        df['住所コード'] = '不明'
        df['住所コード'] = df['住所コード'].astype('category')


    return df

# Modify the main pipeline to use create_advanced_features
def final_pipeline_v2(train_df, test_df):
    """最終調整パイプライン (v2 - 拡張特徴量使用)"""
    print("🚀 最終調整パイプライン（v2 - 拡張特徴量使用）")
    print("="*60)

    # 最終特徴量作成
    train_advanced = create_advanced_features(train_df.copy())
    test_advanced = create_advanced_features(test_df.copy())

    # Define the feature set to use for modeling - Include all new features
    # Let's be explicit about which columns to use as features.
    # Start with a base set and add the newly created ones.

    base_features = [
        '時価価額', '顧客ランク', '年齢×資産規模', '過去取引回数',
        '資産規模', '投資効率', '年齢調整資産', '含み損益率',
        '商品多様性', 'デジタル利用率', '投資成功体験', '過去累積損益',
        'リスク管理度', 'リスク指向度', '円建て比率', '留意商品比率',
        '顧客年齢', '投資経験（株式）', 'year', 'month', '投資方針', # Keep 投資方針 for encoding
        '高資産フラグ', '含み損フラグ', 'シニアフラグ', '株式経験あり'
    ]

    advanced_features_list = [
        '投資効率×年齢', 'リスク×経験', '資産×取引頻度',
        '成功率', '効率ランク', '年齢内資産順位', '総合投資力',
        '異常高資産', '超アクティブ',
        '売却損益変動性', '償還損益変動性', '総損益変動性', '直近取引からの日数',
        '平均リスクカテゴリ_数値', '保有商品数_ユニーク', '最終償還日までの日数',
        '投資経験_損益_大幅損失', '投資経験_損益_軽微損失', '投資経験_損益_軽微利益', '投資経験_損益_大幅利益',
        '投資方針_1_投資経験あり', '投資方針_2_投資経験あり', '投資方針_3_投資経験あり',
        '損失回復期待フラグ', '利益確定後再投資フラグ'
        # Add age_group x profit_loss interactions explicitly if needed, or let get_dummies handle them if they are kept as categorical
        # Let's add the specific interaction columns created
    ]

    # Add age_group x profit_loss interaction column names
    age_categories = ['under40歳', '40_50歳', '50_60歳', '60_70歳', '70歳~', '不明'] # Use cleaned names
    profit_loss_statuses = ['大幅損失', '軽微損失', '軽微利益', '大幅利益', 'その他']
    for age_group in age_categories:
        for profit_loss in profit_loss_statuses:
            col_name = f'年齢層_{age_group}_損益_{profit_loss}' # Use cleaned names
            advanced_features_list.append(col_name)

    # Add 資産規模_区分 and 住所コード to the list of categorical features for encoding
    categorical_to_encode_list = ['投資方針', '投資家タイプ', '主要リスクカテゴリ', '主要商品カテゴリ', '資産規模_区分', '住所コード'] # Added 資産規模_区分 and 住所コード


    final_features_list = list(set(base_features + advanced_features_list)) # Combine and remove duplicates

    # Separate categorical features that need one-hot encoding
    categorical_cols_for_encoding = [col for col in categorical_to_encode_list if col in final_features_list]

    # Final numerical/binary features (excluding those to be encoded)
    numeric_features = [col for col in final_features_list if col not in categorical_cols_for_encoding]


    print(f"📊 数値/バイナリ特徴量: {len(numeric_features)}個")
    print(f"📊 エンコード対象カテゴリ特徴量: {len(categorical_cols_for_encoding)}個")
    # print(f"例 (数値/バイナリ): {numeric_features[:5]}...")
    # print(f"例 (カテゴリ): {categorical_cols_for_encoding}")


    # Term別予測
    all_predictions = pd.DataFrame()
    term_aucs = []

    for term_id in range(1, 7):
        print(f"\n{'='*40}")
        print(f"Term {term_id} 最終調整 (v2)")
        print(f"{'='*40}")

        # データ準備
        train_term = train_advanced[train_advanced[f'train_term_{term_id}'] == 1].copy() # Use advanced features df
        test_term = test_advanced[test_advanced[f'test_term_{term_id}'] == 1].copy() # Use advanced features df

        # Select features - ensure they exist in the dataframe
        X_train_numeric = train_term[[col for col in numeric_features if col in train_term.columns]].fillna(0)
        X_test_numeric = test_term[[col for col in numeric_features if col in test_term.columns]].fillna(0)

        X_train_categorical = train_term[[col for col in categorical_cols_for_encoding if col in train_term.columns]]
        X_test_categorical = test_term[[col for col in categorical_cols_for_encoding if col in test_term.columns]]


        # Categorical encoding
        if not X_train_categorical.empty:
            X_train_categorical_encoded = pd.get_dummies(X_train_categorical, columns=X_train_categorical.columns, drop_first=True, dtype=int)
            X_test_categorical_encoded = pd.get_dummies(X_test_categorical, columns=X_test_categorical.columns, drop_first=True, dtype=int)

            # Combine numeric and encoded categorical features
            X_train = pd.concat([X_train_numeric, X_train_categorical_encoded], axis=1)
            X_test = pd.concat([X_test_numeric, X_test_categorical_encoded], axis=1)

            # Align columns after encoding - crucial for consistent feature sets
            train_cols = X_train.columns
            test_cols = X_test.columns

            missing_in_test = set(train_cols) - set(test_cols)
            for col in missing_in_test:
                X_test[col] = 0

            missing_in_train = set(test_cols) - set(train_cols)
            for col in missing_in_train:
                X_train[col] = 0 # Should not happen with drop_first=True and consistent data

            X_test = X_test[train_cols] # Ensure column order is the same

        else: # No categorical columns to encode
            X_train = X_train_numeric
            X_test = X_test_numeric


        y_train = train_term['y']


        print(f"最終特徴量数 (エンコーディング後): {X_train.shape[1]}個")

        # Scaling numerical features (Optional but often good practice)
        # scaler = StandardScaler()
        # numeric_cols_to_scale = X_train_numeric.columns # Scale only the original numeric features
        # if not numeric_cols_to_scale.empty:
        #      X_train[numeric_cols_to_scale] = scaler.fit_transform(X_train[numeric_cols_to_scale])
        #      X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])
        # Note: Skipping scaling for tree-based models, but leaving comment as reminder.

        # 最終モデル学習 (Use the same ensemble structure)
        ensemble = FinalEnsemble() # Re-initialize for each term
        cv_scores = ensemble.fit(X_train, y_train, term_id)
        term_aucs.append(np.mean(list(cv_scores.values())))

        # 予測
        test_pred = ensemble.predict_proba(X_test)

        # 結果保存
        pred_df = pd.DataFrame({'ID': test_term['ID'], 'predict': test_pred})
        all_predictions = pd.concat([all_predictions, pred_df]) if not all_predictions.empty else pred_df

    # 最終結果
    avg_auc = np.mean(term_aucs)
    print(f"\n{'='*60}")
    print("🎯 最終調整結果 (v2 - 拡張特徴量使用)")
    print(f"{'='*60}")

    for i, auc in enumerate(term_aucs, 1):
        print(f"Term {i}: {auc:.4f}")

    print(f"\n📈 最終CV-AUC: {avg_auc:.4f}")
    # Adjust predicted public score based on typical train/test gap
    predicted_public_auc = avg_auc - 0.027 # Maintain the same adjustment factor
    print(f"📊 予想パブリック: {predicted_public_auc:.4f}")


    # Check against target AUCs
    if avg_auc >= 0.677:
        print("🎉 0.65達成可能性大！")
    elif avg_auc >= 0.67:
        print("✅ 0.64後半達成！")
    else:
        print("📈 特徴量/モデル調整を続ける価値あり。")


    return all_predictions, avg_auc


# Test the new function and pipeline
print("🧪 テスト実行: create_advanced_features")
# Ensure necessary dataframes (trading_df, product_df) and flag (HAS_ALL_DATA) are available
# They should be from previous successful runs.

train_advanced_test = create_advanced_features(train_df.copy())
test_advanced_test = create_advanced_features(test_df.copy())

print(f"\nTrain Advanced shape: {train_advanced_test.shape}")
print(f"Test Advanced shape: {test_advanced_test.shape}")
print("\nTrain Advanced Columns:")
display(train_advanced_test.columns.tolist())
print("\nTest Advanced Columns:")
display(test_advanced_test.columns.tolist())

# Check for new columns (relative to original train_df)
original_train_cols = train_df.columns.tolist()
new_cols = [col for col in train_advanced_test.columns if col not in original_train_cols]
print(f"\nNewly added features count: {len(new_cols)}")
print(f"Newly added features: {new_cols}")

# Check for NaNs in new columns (should be filled by the function)
print("\nChecking NaNs in new columns:")
display(train_advanced_test[new_cols].isnull().sum().sort_values(ascending=False).head())
display(test_advanced_test[new_cols].isnull().sum().sort_values(ascending=False).head())

print("\n🧪 テスト完了: create_advanced_features")

🧪 テスト実行: create_advanced_features

Train Advanced shape: (90000, 111)
Test Advanced shape: (6000, 107)

Train Advanced Columns:


['ID',
 '顧客ID',
 '住所コード',
 '顧客氏名',
 '基準年月',
 '取得価額',
 '時価価額',
 '評価損益',
 '投資方針',
 'train_term_1',
 'train_term_2',
 'train_term_3',
 'train_term_4',
 'train_term_5',
 'train_term_6',
 '翌月_購入',
 '翌月_売却',
 '翌々月_購入',
 '翌々月_売却',
 'y',
 '顧客年齢',
 '投資経験（株式）',
 'year',
 'month',
 '資産規模_区分',
 '投資効率',
 '資産規模',
 '年齢調整資産',
 '顧客ランク',
 '年齢×資産規模',
 '含み損益率',
 '過去取引回数',
 '過去総取得額',
 '過去平均取得額',
 '売却損益合計',
 '売却回数',
 '売却損益変動性',
 '償還損益合計',
 '償還回数',
 '償還損益変動性',
 'デジタル利用率',
 '商品多様性',
 'ゴール設定率',
 'ロスカット設定率',
 '過去累積損益',
 '投資成功体験',
 '平均利益率',
 'リスク管理度',
 '総損益変動性',
 '直近取引からの日数',
 '投資家タイプ',
 '主要商品カテゴリ',
 '主要リスクカテゴリ',
 '平均リスクカテゴリ_数値',
 '円建て比率',
 '留意商品比率',
 '保有商品数_ユニーク',
 'リスク指向度',
 '最終償還日までの日数',
 '高資産フラグ',
 '含み損フラグ',
 'シニアフラグ',
 '株式経験あり',
 '投資効率×年齢',
 'リスク×経験',
 '資産×取引頻度',
 '成功率',
 '効率ランク',
 '年齢内資産順位',
 '総合投資力',
 '異常高資産',
 '超アクティブ',
 '投資経験_損益_大幅損失',
 '投資経験_損益_軽微損失',
 '投資経験_損益_軽微利益',
 '投資経験_損益_大幅利益',
 '年齢層_under40歳_損益_大幅損失',
 '年齢層_under40歳_損益_軽微損失',
 '年齢層_under40歳_損益_軽微利益',
 '年齢層_under40歳_損益_大幅利益',
 '年齢層_under40歳_損益_そ


Test Advanced Columns:


['Unnamed: 0',
 'ID',
 '顧客ID',
 '住所コード',
 '顧客氏名',
 '基準年月',
 '取得価額',
 '時価価額',
 '評価損益',
 '投資方針',
 'test_term_1',
 'test_term_2',
 'test_term_3',
 'test_term_4',
 'test_term_5',
 'test_term_6',
 '顧客年齢',
 '投資経験（株式）',
 'year',
 'month',
 '資産規模',
 '投資効率',
 '年齢調整資産',
 '顧客ランク',
 '年齢×資産規模',
 '含み損益率',
 '過去取引回数',
 '過去総取得額',
 '過去平均取得額',
 '売却損益合計',
 '売却回数',
 '売却損益変動性',
 '償還損益合計',
 '償還回数',
 '償還損益変動性',
 'デジタル利用率',
 '商品多様性',
 'ゴール設定率',
 'ロスカット設定率',
 '過去累積損益',
 '投資成功体験',
 '平均利益率',
 'リスク管理度',
 '総損益変動性',
 '直近取引からの日数',
 '投資家タイプ',
 '主要商品カテゴリ',
 '主要リスクカテゴリ',
 '平均リスクカテゴリ_数値',
 '円建て比率',
 '留意商品比率',
 '保有商品数_ユニーク',
 'リスク指向度',
 '最終償還日までの日数',
 '高資産フラグ',
 '含み損フラグ',
 'シニアフラグ',
 '株式経験あり',
 '投資効率×年齢',
 'リスク×経験',
 '資産×取引頻度',
 '成功率',
 '効率ランク',
 '年齢内資産順位',
 '総合投資力',
 '異常高資産',
 '超アクティブ',
 '投資経験_損益_大幅損失',
 '投資経験_損益_軽微損失',
 '投資経験_損益_軽微利益',
 '投資経験_損益_大幅利益',
 '年齢層_under40歳_損益_大幅損失',
 '年齢層_under40歳_損益_軽微損失',
 '年齢層_under40歳_損益_軽微利益',
 '年齢層_under40歳_損益_大幅利益',
 '年齢層_under40歳_損益_その他',
 '年齢層_40_50歳_損益_大幅損失',
 '年齢層_40_50歳_損益_軽微損失',



Newly added features count: 85
Newly added features: ['資産規模', '年齢調整資産', '顧客ランク', '年齢×資産規模', '含み損益率', '過去取引回数', '過去総取得額', '過去平均取得額', '売却損益合計', '売却回数', '売却損益変動性', '償還損益合計', '償還回数', '償還損益変動性', 'デジタル利用率', '商品多様性', 'ゴール設定率', 'ロスカット設定率', '過去累積損益', '投資成功体験', '平均利益率', 'リスク管理度', '総損益変動性', '直近取引からの日数', '投資家タイプ', '主要商品カテゴリ', '主要リスクカテゴリ', '平均リスクカテゴリ_数値', '円建て比率', '留意商品比率', '保有商品数_ユニーク', 'リスク指向度', '最終償還日までの日数', '高資産フラグ', '含み損フラグ', 'シニアフラグ', '株式経験あり', '投資効率×年齢', 'リスク×経験', '資産×取引頻度', '成功率', '効率ランク', '年齢内資産順位', '総合投資力', '異常高資産', '超アクティブ', '投資経験_損益_大幅損失', '投資経験_損益_軽微損失', '投資経験_損益_軽微利益', '投資経験_損益_大幅利益', '年齢層_under40歳_損益_大幅損失', '年齢層_under40歳_損益_軽微損失', '年齢層_under40歳_損益_軽微利益', '年齢層_under40歳_損益_大幅利益', '年齢層_under40歳_損益_その他', '年齢層_40_50歳_損益_大幅損失', '年齢層_40_50歳_損益_軽微損失', '年齢層_40_50歳_損益_軽微利益', '年齢層_40_50歳_損益_大幅利益', '年齢層_40_50歳_損益_その他', '年齢層_50_60歳_損益_大幅損失', '年齢層_50_60歳_損益_軽微損失', '年齢層_50_60歳_損益_軽微利益', '年齢層_50_60歳_損益_大幅利益', '年齢層_50_60歳_損益_その他', '年齢層_60_70歳_損益_大幅損失', '年齢層_60_70歳_損益_軽微損失', '年齢層_60_70歳_損益_軽微利益', '年齢

,0
資産規模,0
年齢調整資産,0
顧客ランク,0
年齢×資産規模,0
含み損益率,0


,0
資産規模,0
年齢調整資産,0
顧客ランク,0
年齢×資産規模,0
含み損益率,0



🧪 テスト完了: create_advanced_features


In [ ]:
# 🚀 最終調整実行（v2 - 拡張特徴量使用）
print("🚀 最終調整実行（v2 - 拡張特徴量使用）")
# Ensure train_df and test_df are available from previous cells
if 'train_df' in locals() and 'test_df' in locals():
    final_predictions_v2, final_auc_v2 = final_pipeline_v2(train_df, test_df)

    # 提出ファイル作成 (using the results from the v2 pipeline)
    submission_df_v2 = pd.read_csv(base_dir + '/sample_submit.csv', header=None) # Load sample submission again
    # Ensure predictions are aligned by ID before assigning
    final_predictions_v2 = final_predictions_v2.sort_values('ID')
    submission_df_v2[0] = final_predictions_v2['ID'].values # Ensure IDs match
    submission_df_v2[1] = final_predictions_v2['predict'].values
    submission_df_v2.to_csv(base_dir + '/final_submission_v2.csv', index=False, header=False)


    print(f"\n🎯 最終調整完了 (v2)！")
    print(f"CV-AUC: {final_auc_v2:.4f}")
    # Ensure final_auc is available from previous runs for comparison, or calculate/estimate it.
    # Assuming final_auc from the previous successful run (0.6694) is accessible.
    # If not, just print the new predicted public AUC.
    try:
        # Access the previously calculated final_auc from the old pipeline run
        print(f"予想改善: {final_auc - 0.027:.4f} (旧) → {final_auc_v2 - 0.027:.4f} (新)") # Compare predicted public AUC
    except NameError:
         print(f"予想パブリック (新): {final_auc_v2 - 0.027:.4f}") # Print only new if old AUC is not available

    print("提出ファイル: final_submission_v2.csv")

    # The subtask is completed as the new features are implemented and used in the pipeline execution.
    # Report success and relevant dataframes.
    # Note: final_pipeline_v2 uses train_advanced and test_advanced internally, but they are not returned.
    # The relevant dataframes for the output of the subtask are train_df and test_df, which were used as input.
    # Also include the submission dataframe.


else:
    print("❌ Error: train_df or test_df not found. Cannot run pipeline.")

🚀 最終調整実行（v2 - 拡張特徴量使用）
🚀 最終調整パイプライン（v2 - 拡張特徴量使用）
📊 数値/バイナリ特徴量: 79個
📊 エンコード対象カテゴリ特徴量: 1個

Term 1 最終調整 (v2)
最終特徴量数 (エンコーディング後): 76個

🎯 Term 1 - 最終調整学習
データ: (80000, 76), 陽性率: 0.1972
  rf1: 0.7428 (±0.003)
  et1: 0.7543 (±0.001)
  rf2: 0.7183 (±0.003)
  最終重み: {'rf1': np.float64(0.3373366697497225), 'et1': np.float64(0.34918922306038613), 'rf2': np.float64(0.3134741071898914)}

Term 2 最終調整 (v2)
最終特徴量数 (エンコーディング後): 76個

🎯 Term 2 - 最終調整学習
データ: (82000, 76), 陽性率: 0.1976
  rf1: 0.7421 (±0.004)
  et1: 0.7534 (±0.004)
  rf2: 0.7167 (±0.004)
  最終重み: {'rf1': np.float64(0.3377126864833012), 'et1': np.float64(0.34935847614966664), 'rf2': np.float64(0.31292883736703214)}

Term 3 最終調整 (v2)
最終特徴量数 (エンコーディング後): 76個

🎯 Term 3 - 最終調整学習
データ: (84000, 76), 陽性率: 0.1980
  rf1: 0.7403 (±0.001)
  et1: 0.7526 (±0.002)
  rf2: 0.7157 (±0.002)
  最終重み: {'rf1': np.float64(0.33712460022727575), 'et1': np.float64(0.34976384246186165), 'rf2': np.float64(0.3131115573108626)}

Term 4 最終調整 (v2)
最終特徴量数 (エンコーディング後): 76個

🎯 Ter

## 新しい特徴量のアイデア出し

### Subtask:
特定した顧客層や傾向に基づき、さらにモデルの予測力を高める可能性のある特徴量のアイデアを具体的にリストアップします。例えば、過去の取引データや商品リストをより詳細に分析して得られる情報や、既存の特徴量を組み合わせた交互作用項などが考えられます。